* Import necessary libraries.

In [ ]:
import pandas as pd

from fuzzywuzzy import fuzz, process

* Read dataset.
* Lines 3350, 4704, 5879 and 8981 contain 13 values instead of 12, so they are considered problematic. (Book IDs: 12224, 16914, 22128, 34889)
* Considering they were only 4 lines, I changed them manually.
* Also, lines 8182 and 11100 contain invalid dates, so they are considered problematic. (Book IDs: 31371, 45531)
* Considering they were only 2 lines, I changed them manually.

In [2]:
df = pd.read_csv('books.csv')

* Upon Inspecting the data, I found 0 null values and 0 duplicates.

In [3]:
# Count null values
print('Null values:', df.isnull().sum().sum())

# Count duplicates
print('Duplicates:', df.duplicated().sum())

Null values: 0
Duplicates: 0


* Check the column data types.

In [4]:
print(df.dtypes)

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
num_pages               int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object


* Standardize author names.

In [5]:
# Store the original 'author' column for comparison
original_authors = df['authors'].copy()

# Standardize the author names
df['authors'] = df['authors'].str.strip()  # Remove leading/trailing spaces
df['authors'] = df['authors'].str.replace(r'\s+', ' ', regex=True)  # Replace multiple spaces with a single space
df['authors'] = df['authors'].str.title()  # Convert to title case

# Count how many rows were changed
changed_rows = (df['authors'] != original_authors).sum()

print(f'Total number of changed rows: {changed_rows}')

Total number of changed rows: 1128


* We can see that there are instances where the language is en-US, en-GB or en-CA instead of eng, which in this analysis is not necessary.
* We map these three to eng so they are the same.

In [6]:
df['language_code'].value_counts()

language_code
eng      8911
en-US    1409
spa       218
en-GB     214
fre       144
ger        99
jpn        46
mul        19
zho        14
grc        11
por        10
en-CA       7
ita         5
enm         3
lat         3
swe         2
rus         2
srp         1
nl          1
msa         1
glg         1
wel         1
ara         1
nor         1
tur         1
gla         1
ale         1
Name: count, dtype: int64

In [7]:
df['language_code'] = df['language_code'].replace({
    'en-US': 'eng',
    'en-GB': 'eng',
    'en-CA': 'eng'
})

* We also can see some issues on the publishers.
* We map all publisher values with issues to the proper name.
* Because the fuzzing takes some time it is all a comment.
* Also, its results are lengthy, so they are written in markdown under the analysis.

In [8]:
# # Function to find similar publisher names
# def find_similar_publishers(df, column_name, threshold=80):
#     publishers = df[column_name].unique()
#     similar_publishers = {}
    
#     for publisher in publishers:
#         matches = process.extract(publisher, publishers, limit=None, scorer=fuzz.token_sort_ratio)
#         # Filter matches above the similarity threshold
#         matches_above_threshold = [match for match in matches if match[1] >= threshold and match[0] != publisher]
#         if matches_above_threshold:
#             similar_publishers[publisher] = matches_above_threshold
    
#     return similar_publishers

# # Find similar publishers
# similar_publishers = find_similar_publishers(df, 'publisher')

# # Print results
# for publisher, matches in similar_publishers.items():
#     print(f'Publisher: {publisher}')
#     for match in matches:
#         print(f'  - Similar: {match[0]} (Similarity: {match[1]}%)')

In [9]:
df['publisher'] = df['publisher'].replace({
    'Scholastic  Inc.': 'Scholastic Inc.',
    'Scholastic Inc': 'Scholastic Inc.',
    'Scholastic': 'Scholastic Inc.',
    'Scholastic Paperbacks': 'Scholastic Inc.',
    'Apple Paperbacks (Scholastic)': 'Scholastic Inc.',
    'Scholastic Books': 'Scholastic Inc.',
    'Scholastic Audio Books': 'Scholastic Inc.',
    
    'Del Rey Books (NY)': 'Del Rey Books',
    
    'Random House Anchor': 'Random House',
    'Random House Audio': 'Random House',
    'Random house': 'Random House',
    'Random House for Young Readers': 'Random House',
    'Random House  for Young Readers': 'Random House',
    'Random House Books for Young Readers': 'Random House',
    'Random House (NY)': 'Random House',
    'Random House  Inc.': 'Random House',
    'Random House Inc.': 'Random House',
    'Random House Trade': 'Random House',
    'Random House Large Print Publishing': 'Random House',
    'Random House Large Print': 'Random House',
    'Random House Vintage': 'Random House',
    'Random House (Vintage)': 'Random House',
    'Random House Vintage Books': 'Random House',
    'Vintage/Random House (NY)': 'Random House',
    'Vintage/Random House Inc. (NYC)': 'Random House',
    'Random House Children\'s Books': 'Random House',
    'Random House Value Publishing': 'Random House',
    'Random House Reference Publishing': 'Random House',
    
    'Random House: Modern Library': 'Modern Library/Random House',
    'Modern Library/Random House (NY)': 'Modern Library/Random House',   

    'BALLANTINE BOOKS': 'Ballantine Books',
    'Ballantine Books (NY)': 'Ballantine Books',
    
    'Houghton Mifflin Harcourt P': 'Houghton Mifflin',
    'Houghton Mifflin Harcourt': 'Houghton Mifflin',
    'Houghton Mifflin Company': 'Houghton Mifflin',
    
    'New Press  The': 'The New Press',
    'New Press': 'The New Press',
    'New Press (NY)': 'The New Press',
    
    'Universe': 'iUniverse',
    
    'Farrar Straus and Giroux': 'Farrar Straus and Giroux',
    'Farrar  Straus and Giroux (NY)': 'Farrar Straus and Giroux',
    'Farrar  Straus and Giroux (BYR)': 'Farrar Straus and Giroux',
    'Farrar Straus Giroux': 'Farrar Straus and Giroux',
    'Farrar  Straus & Giroux': 'Farrar Straus and Giroux',
    'Farrar  Strauss & Giroux-3pl': 'Farrar Straus and Giroux',
    
    'Dramatists Play Service': 'Dramatists Play Service Inc.',
    
    'Penguin Books (NY)': 'Penguin Books',
    'Penguin Books Ltd': 'Penguin Books',
    'Penguin Books (NYC)': 'Penguin Books',
    'Penguin Books Ltd.': 'Penguin Books',
    'Penguin Classic': 'Penguin Books',
    'Penguin (Non-Classics)': 'Penguin Books',
    'Penguin': 'Penguin Books',
    'Penguin UK': 'Penguin Books',
    'Penguin Books (London)': 'Penguin Books',
    'Penguin Books Ltd. (London)': 'Penguin Books',
    'Penguin Books Limited': 'Penguin Books',
    'Penguin Book Limited': 'Penguin Books',
    'Plume (Penguin Books Ltd': 'Penguin Books',
    'Penguin Books  Limited (UK)': 'Penguin Books',
    'Penguin Books Canada': 'Penguin Books',
    'Penguin Books (Penguin Classics)': 'Penguin Books',
    'Penguin Books /Penguin Classics': 'Penguin Books',
    'Penguin Books/Penguin Popular Classics': 'Penguin Books',
    'Penguin Audio': 'Penguin Books',
    'Penguin Audio UK': 'Penguin Books',
    'Penguin Classics Deluxe Editions': 'Penguin Books',
    'Penguin Classics Deluxe Edition': 'Penguin Books',
    'Penguin Books USA Inc.': 'Penguin Books',
    'Penguin Group': 'Penguin Books',
    'Penguin Group(CA)': 'Penguin Books',
    'Penguin Group (USA)': 'Penguin Books',
    
    'Barnes  Noble Classics': 'Barnes & Noble',
    'Barnes  Noble': 'Barnes & Noble',
    'Barnes & Noble Books': 'Barnes & Noble',
    
    'Dover Publications (NY)': 'Dover Publications',
    'Dover Publications  Inc.': 'Dover Publications',
    'Dover Publications (NYC)': 'Dover Publications',
    
    'HarperPerennial': 'Harper Perennial',
    'Harper Perennial (NYC)': 'Harper Perennial',
    'HarperCollins Perennial': 'Harper Perennial',
    'Harper Perennial (HarperCollins)': 'Harper Perennial',
    'Harper Perennial Classics': 'Harper Perennial',
    'HarperPerennial / Perennial Classics': 'Harper Perennial',
    'Harper Perennial Modern Classics': 'Harper Perennial',
    
    'New Directions Publishing Corporation': 'New Directions Publishing',
    
    'Grove Press (NYC)': 'Grove Press',
    
    'Simon  Schuster': 'Simon & Schuster',
    'Simon Schuster': 'Simon & Schuster',
    'Simon and Schuster': 'Simon & Schuster',
    'Simon  Schuster Audio': 'Simon & Schuster',
    'Simon & Schuster Audio': 'Simon & Schuster',
    'Simon & Schuster Aladddin Paperbacks': 'Simon & Schuster',
    'Simon & Schuster Paperbacks': 'Simon & Schuster',
    'Simon  Schuster Books for Young Readers': 'Simon & Schuster',
    'Simon & Schuster Books for Young Readers': 'Simon & Schuster',
    
    'French & European Pubns': 'French & European Publications',
    'French & European': 'French & European Publications',
    
    'Samuel French Ltd': 'Samuel French Inc.',
    
    'Princeton University Press (NJ)': 'Princeton University Press',
    
    'ACE': 'Ace Books',
    'Ace Book': 'Ace Books',
    'Ace Hardcover': 'Ace Books',
    
    'Tommy Nelson': 'Thomas Nelson',
    
    'W. W. Norton  Company': 'W. W. Norton & Company',
    'W.W. Norton & Company': 'W. W. Norton & Company',
    'W.W. Norton  Company': 'W. W. Norton & Company',
    'W.W. Norton & Company (NY)': 'W. W. Norton & Company',
    'W.W. Norton & Company (NYC)': 'W. W. Norton & Company',
    'W. W. Norton and Company': 'W. W. Norton & Company',
    'W. W. Norton & Co.': 'W. W. Norton & Company',
    'W.W. Norton & Company  Inc. (NY)': 'W. W. Norton & Company',
    'W.W. Norton': 'W. W. Norton & Company',
    'W.W. Norton & Company (NY/London)': 'W. W. Norton & Company',
    
    'Everyman\'s Library': 'Everymans Library',
    'Everyman\'s Library 234': 'Everymans Library',
    'Everyman\'s Library Classics': 'Everymans Library',
    
    'Sun & Moon Press': 'Sun and Moon Press',
    
    'Henry Holt and Company': 'Henry Holt & Company',
    'Henry Holt and Co. (BYR)': 'Henry Holt & Company',
    'Henry Holt and Co.': 'Henry Holt & Company',
    
    'Listening Library (Audio)': 'Listening Library',
    'Listening Library': 'Listening Library',
    
    'Crown Publishing Group (NY)': 'Crown Publishing Group',
    
    'Hackett Publishing Company  Inc. (USA)': 'Hackett Publishing Company Inc.',
    'Hackett Publishing Company': 'Hackett Publishing Company Inc.',
    'Hackett Publishing Company (Indianapolis  IN)': 'Hackett Publishing Company Inc.',
    
    'Bloomsbury Publishing PLC': 'Bloomsbury Publishing',
    
    'The Berkley Publishing Group': 'Berkley Publishing Group',
    'Berkley Publishing': 'Berkley Publishing Group',
    
    'Orion Publishing Group  Ltd.': 'Orion Publishing Group',
    'Orion Publishing Co.': 'Orion Publishing Group',
    'Orion Publishing Co': 'Orion Publishing Group',
    'Orion Publishing': 'Orion Publishing Group',
    'The Orion Publishing Group Ltd': 'Orion Publishing Group',
    
    'Plexus Publishing Ltd': 'Plexus Publishing',
    
    'Green Ronin Publishing': 'Ronin Publishing (CA)',
    
    'Arcadia Publishing (SC)': 'Arcadia Publishing',
    
    'Crossroad Publishing Company': 'The Crossroad Publishing Company',
    
    'Bantam Doubleday Dell Publishing Group': 'Bantam Doubleday Dell Publishing Company Inc.',
    
    'Zondervan Publishing House': 'Zondervan Publishing Company',
    
    'Wiley Publishing  Inc.': 'Wiley Publishing',
    'Wiley Publishing': 'Wiley Publishing',
    
    'Oxford University Press  USA': 'Oxford University Press',
    
    'University Of Chicago Press': 'University of Chicago Press',
    
    'Harvard University Press (Cambridge)': 'Harvard University Press',
    
    'Univ Of Minnesota Press': 'University of Minnesota Press',
    
    'Northwestern Univ Press': 'Northwestern University Press',
    
    'Stanford University Press (Stanford  CA)': 'Stanford University Press',
    
    'Kent State University Press (OH)': 'Kent State University Press',
    
    'Prentice Hall PTR': 'Prentice Hall Press',
    'Prentice Hall': 'Prentice Hall Press',
    
    'Three Rivers Press (CA)': 'Three Rivers Press',
    
    'MIT Press (MA)': 'MIT Press',
    'The MIT Press': 'MIT Press',
    
    'Candlewick Press (MA)': 'Candlewick Press',
    
    'Harvill Press': 'The Harvill Press',
    
    'St. Martin\'s Press': 'St. Martins Press',
    'St. Martins Press-3PL': 'St. Martins Press',
    
    'Atlantic Monthly Press (NYC)': 'Atlantic Monthly Press',
    
    'Running Press Kids': 'Running Press',
    'Running Press Adult': 'Running Press',
    
    'Phoenix Press (UK)': 'Phoenix Press',
    
    'Inter-Varsity Press': 'Inter Varsity Press',
    
    'Graphics Press LLC': 'Graphics Press',
    'Graphics Press': 'Graphics Press',
    
    'The Overlook Press': 'Overlook Press',
    'Overlook TP': 'Overlook Press',
    
    'South End Press (Boston)': 'South End Press',
    
    'HarperCollins Publishers': 'HarperCollinsPublishers',
    'HarperCollins Publishers Inc.': 'HarperCollinsPublishers',
    'HarperCollins Publishers Ltd': 'HarperCollinsPublishers',
    'Collins Publishers': 'HarperCollinsPublishers',
    'HarperCollins': 'HarperCollinsPublishers',
    'Harpercollins': 'HarperCollinsPublishers',
    'HarperCollins UK': 'HarperCollinsPublishers',
    'HarperCollins Publishers': 'HarperCollinsPublishers',
    'HarperCollins Espanol': 'HarperCollinsPublishers',
    'Harpercollins Children\'s Books': 'HarperCollinsPublishers',
    'Harpercollins Childrens Books': 'HarperCollinsPublishers',
    'HarperCollins (SanFrancisco)': 'HarperCollinsPublishers',
    'HarperSanFrancisco': 'HarperCollinsPublishers',
    
    'William Morrow / HarperCollins / Harper Perennial': 'Perennial / William Morrow / HarperCollins',
    
    'The New York Review Children\'s Collection': 'New York Review Children\'s Collection',
    
    'MacMillan': 'Macmillan',
    'Macmillan UK': 'Macmillan',
    'Pan MacMillan': 'Macmillan',
    'MacMillan Children\'s Books': 'Macmillan',
    'Macmillan Children\'s Books': 'Macmillan',
    'Macmillan Audio': 'Macmillan',
    'Macmillan Audio': 'Macmillan',
    
    'Bloomsbury': 'Bloomsbury',
    'Bloomsbury UK': 'Bloomsbury',
    'Bloomsbury USA': 'Bloomsbury',
    'Bloomsbury (NYC)': 'Bloomsbury',
    'Bloomsbury Childrens Books': 'Bloomsbury',
    'Bloomsbury Children\'s Books': 'Bloomsbury',
    'Bloomsbury USA Children\'s Books': 'Bloomsbury',
    'Bloomsbury U.S.A. Children\'s Books': 'Bloomsbury',
    'Bloomsbury USA Childrens': 'Bloomsbury',
    
    'Signet Book': 'Signet Classics',
    'Signet Classic': 'Signet Classics',
    'Signet Books (NY)': 'Signet Books',
    
    'Alfred A. Knopf': 'Alfred A.Knopf',
    'Alfred A.Knopf': 'Alfred A.Knopf',
    'Alfred A. Knopf Inc.': 'Alfred A. Knopf',
    'Alfred A. Knopf Books for Young Readers': 'Knopf Books for Young Readers',
    'Knopf Books for Young Readers': 'Knopf Books for Young Readers',
    
    'Plaza y Janes': 'Plaza y Janés',
    'Plaza & Janés': 'Plaza y Janés',
    'Plaza & Janes Editores Sa': 'Plaza & Janés Editores S.A.',
    'Plaza & Janés Editores S.A.': 'Plaza & Janés Editores S.A.',
    
    'Vintage Books': 'Vintage Books USA',
    
    'Bantam Books': 'Bantam Books (NY)',
    'Bantam Books (NY)': 'Bantam Books',
    'Bantam Books Inc.': 'Bantam Books',
    'Bantam Classics': 'Bantam Classic',
    'Bantam Classic': 'Bantam Classic',
    
    'Spectra/Bantam Books': 'Bantam Spectra (NY)',
    'Bantam Spectra (NY)': 'Bantam Spectra',
    'Bantam Spectra': 'Bantam Spectra',
    'BantamSpectra': 'Bantam Spectra',
    'Spectra/Bantam Books (NYC)': 'Bantam Spectra',
    
    'Basic Books': 'Basic Books (AZ)',
    
    'VIZ Media LLC': 'VIZ Media',
    'VIZ Media': 'VIZ Media',
    'Viz Media': 'VIZ Media',
    
    'Amherst Media': 'Amherst Media Inc.',
    
    'Moody Publishers': 'Moody Publishers (Chicago)',
    
    'Clarkson Potter Publishers': 'Clarkson Potter/Publishers',
    
    'Peter Smith Publisher': 'Peter Smith Publisher Inc.',
    
    'International Publishers': 'International Publishers (NYC)',
    
    'Chelsea House Publications': 'Chelsea House Publishers',
    
    'Ivan R. Dee Publisher': 'Ivan R. Dee Publisher',
    
    'Transworld Publishers': 'Transworld Publishers Ltd',
    
    'Marion Boyars Publishers': 'Marion Boyars Publishers Ltd',
    
    'Sage Publications Ltd': 'Sage Publications Inc',
    
    'Teacher\'s Pet Publications': 'Teacher\'s Pet Publications Inc.',
    
    'Berkley Books': 'Berkley Books (NY)',
    
    'Recorded Books Inc.': 'Recorded Books',
    'Recorded Books': 'Recorded Books',
    
    'Warner Books': 'Warner Books (NY)',
    'Warner Books (NY)': 'Warner Books',
    'TIme Warner Books': 'Warner Books',
    'Time Warner Books UK': 'Warner Books',
    
    'Pantheon Books': 'Pantheon Books (NY)',
    
    'Arthur A. Levine': 'Arthur A. Levine Books',
    
    'Walker Books': 'Walker Books Ltd',
    
    'Avon Books': 'Avon Books (P)',
    
    'Time-Life Books Inc.': 'Time Life Books',
    'Time Life Books': 'Time-Life Books',
    
    'Titan Books Ltd': 'Titan Books',
    'Titan Books': 'Titan Books',
    
    'Orchard Books': 'Orchard Books (NY)',
    
    'DAW Books': 'Daw Books',
    'Daw Books': 'DAW Books',
    'DAW Books (NY)': 'DAW Books',
    'DAW Books Inc': 'DAW Books',
    
    'Arrow Books': 'Arrow Books Ltd',
    
    'Gurze Books': 'Gürze Books',
    
    'Owl Books': 'Owl Books (NY)',
    
    'Laurel-Leaf Books': 'Laurel Leaf Books',
    
    'Fireside Books': 'Firebird Books',
    
    'Disney-Hyperion': 'Disney Hyperion Books',
    'Disney Hyperion Books': 'Disney Hyperion Books',
    'Hyperion Books': 'Disney Hyperion Books',
    
    'Schocken Books': 'Schocken Books Inc',
    
    'Mira Books': 'Mira books',
    
    'BBC Audiobooks': 'BBC Audiobooks Ltd',
    
    'Michael O\'Mara': 'Michael O\'Mara Books',
    
    'Routledge/Taylor & Francis Ltd.': 'Routledge Taylor & Francis Books Ltd imprint',
    
    'Fantagraphics': 'Fantagraphics Books',
    
    'Berkley Prime Crime': 'Berkley Prime Crime Books',
    
    'Fawcett Crest': 'Fawcett crest',
    'Fawcett Crest': 'Fawcett Crest Books',
    
    'Puffin Books': 'Puffin Audiobooks',
    
    'Tom Doherty Associates': 'Doherty Tom Associates LLC',
    
    'TarcherPerigee': 'Tarcherperigee',
    
    'John Wiley': 'John Wiley & Sons',
    
    'Little Brown and Company': 'Little Brown',
    'Little  Brown and Company': 'Little Brown',
    'Little  Brown & Company': 'Little Brown',
    'Little Brown': 'Little Brown',
    'Little  Brown': 'Little Brown',
    
    'Harper': 'Harper Paperbacks',
    'Harper Paperbacks': 'Harper Paperbacks',
    'HarperPaperbacks': 'Harper Paperbacks',
    'HarperTeen': 'Harper Teen',
    'Harper Teen': 'Harper Teen',
    'Harper Voyager': 'HarperVoyager',
    'HarperVoyager': 'HarperVoyager',
    'Harper Trophy': 'HarperTrophy',
    'HarperTrophy': 'HarperTrophy',
    
    'BradyGames': 'Bradygames',
    'Bradygames': 'Bradygames',
    
    'Viking': 'Viking UK',
    
    'Loeb Classical Library': 'Loeb Classical Library',
    'Loeb Classical Library 307': 'Loeb Classical Library',
    'Loeb Classical Library 443': 'Loeb Classical Library',
    
    'Granta': 'Granta UK',
    
    'Collectors Library':'Collector\'s Library',
    
    'The Modern Library': 'Modern Library',
    'Modern Library': 'Modern Library',
    'Modern Library (NY)': 'Modern Library',
    'The Modern Library New York': 'Modern Library',
    
    'Library of America': 'Library of America',
    'Library of America (NY)': 'Library of America',
    
    'G.P. Putnam\'s Sons': 'G. P. Putnam\'s Sons',
    'G. P. Putnam\'s Sons': 'G. P. Putnam\'s Sons',
    
    'Schirmer/Mosel': 'Schirmer Mosel',
    
    'Barrons Educational Series': 'Barron\'s Educational Series',
    'Barron\'s Educational Series': 'Barron\'s Educational Series',
    
    'Health Communications': 'Health Communications Inc',
    
    'Thames & Hudson': 'Thames  Hudson',
    'Thames  Hudson': 'Thames  Hudson',
    'Thames & Hudson Ltd': 'Thames  Hudson',
    
    'David C. Cook': 'David C Cook',
    'David C Cook': 'David C Cook',
    
    'Le Livre de Poche': 'Livre de Poche',
    
    'Faber & Faber': 'Faber  Faber',
    'Faber  Faber': 'Faber  Faber',
    'Faber and Faber': 'Faber  Faber',
    'Faber & Faber Ltd.': 'Faber  Faber',
    
    'William Morrow & Company': 'William Morrow & Company',
    'William Morrow & Company (NYC)': 'William Morrow & Company',
    'William Morrow & Company  Inc.': 'William Morrow & Company',
    
    'William Heinemann': 'William Heinemann',
    'William Heinemann Ltd.': 'William Heinemann',
    
    'Hyperion': 'Hyperion',
    'Hyperio': 'Hyperion',
    'Hiperión': 'Hyperion',
    
    'Harcourt Brace Jovanovich': 'Harcourt Brace Jovanovich',
    'Harcourt  Brace  Jovanovich': 'Harcourt Brace Jovanovich',
    'Harcourt Brace Jovanovich (NY)': 'Harcourt Brace Jovanovich',
    'Harcourt  Brace Jovanovich/Harvest': 'Harcourt Brace Jovanovich',
    
    'Harcourt Brace & Company': 'Harcourt Brace & company',
    'Harcourt Brace & company': 'Harcourt Brace & company',
    
    'Elloras Cave': 'Ellora\'s Cave',
    'Ellora\'s Cave': 'Ellora\'s Cave',
    
    'Counterpoint': 'Counterpoint LLC',
    
    'Audio Partners': 'The Audio Partners',
    
    'Douglas  McIntyre': 'Douglas & McIntyre',
    
    'Hodder & Stoughton Ltd': 'Hodder & Stoughton',
    'Hodder and Stoughton': 'Hodder & Stoughton',
    
    'St. Martin\'s Paperback': 'St. Martin\'s Paperbacks',
    
    'Chatto  Windus': 'Chatto & Windus',
    'Chatto and Windus': 'Chatto & Windus',
    
    'The Chicken House': 'Chicken House',
    
    'Debolsillo': 'DeBolsillo',
    'DEBOLS!LLO': 'DeBolsillo',
    
    'Fourth Estate (GB)': 'Fourth Estate',
    'Fourth Estate Ltd': 'Fourth Estate',
    
    'AudioGO': 'Audiogo',
    
    'J\'ai lu': 'J\'ai Lu',
    'J\'AI LU': 'J\'ai Lu',
    
    'Virago UK': 'Virago',
    
    'McClelland and Stewart': 'McClelland & Stewart',
    
    'Rowohlt Tb.': 'Rowohlt Tb',
    'Rowohlt': 'Rowohlt Tb',
    
    'Games Workshop(uk)': 'Games Workshop',
    
    'Fleming H Revell Co': 'Fleming H. Revell Company',
    
    'Hill and Wang': 'Hill & Wang',
    
    'Éditions du Rocher': 'Editions du Rocher',
    
    'Éditions 10/18': 'Editions 10/18',
    
    'Éditions de L\'Olivier': 'Editions de l\'Olivier',
    
    'Blanvalet Taschenbuch Verlag': 'Taschenbuch Verlag',
    'Blanvalet Taschenbuch': 'Taschenbuch Verlag',
    'Deutscher Taschenbuch Verlag': 'Taschenbuch Verlag',
    'Fischer Taschenbuch Verlag': 'Taschenbuch Verlag',
    
    'George Braziller': 'George Braziller Inc.',
    
    'Tusquets': 'TusQuets',
    
    'Hamish hamilton': 'Hamish Hamilton',
    'Hamish Hamilton Ltd': 'Hamish Hamilton',
    
    'Aris and Phillips': 'Aris & Phillips',
    
    'Eos': 'EOS',
    
    'Hqn': 'HQN',
    
    'Tokyopop': 'TokyoPop',
    
    'Amereon Ltd': 'Amereon Limited',
    
    'Emecé Editores': 'Emece Editores',
    
    'Marvel Comics Group': 'Marvel Comics',
    
    'Jump At The Sun': 'Jump at the Sun',
    
    'Vertigo (DC Comics)': 'DC Comics Vertigo',
    
    'Ullstein Tb': 'Ullstein',
    
    'Fischer (Tb.)': 'Fischer TB',
    
    'dtv': 'Dtv',
    
    'The Disinformation Company (NYC)': 'Disinformation Company',
    
    'Alderac Entertainment Group (AEG)': 'Alderac Entertainment Group',
    
    'MIRA': 'Mira',
    
    'Serpent\'s Tail': 'Serpents Tail',
    
    'Readers Digest.': 'Readers Digest'
    
})

* Split authors and publishers in multiple rows when they are more than 1.

In [10]:
# Split authors and publishers into lists
df['authors'] = df['authors'].str.split('/')
df['publisher'] = df['publisher'].str.split('/')

# Explode authors into multiple rows
df = df.explode('authors')

# Explode publishers into multiple rows
df = df.explode('publisher')

# Strip whitespace from authors and publishers
df['authors'] = df['authors'].str.strip()
df['publisher'] = df['publisher'].str.strip()

* Generate unique id's for author, publisher and language.

In [11]:
df['author_id'] = pd.factorize(df['authors'])[0] + 1
df['language_id'] = pd.factorize(df['language_code'])[0] + 1
df['publisher_id'] = pd.factorize(df['publisher'])[0] + 1

* Save the cleaned data to a new csv.

In [12]:
#df.to_csv('cleaned_books.csv', index=False)

### Appendix

* If needed, the language codes correspond to this:
1. eng: English (general)
2. en-US: English (United States)
3. fre: French
4. spa: Spanish
5. en-GB: English (United Kingdom)
6. mul: Multiple languages (typically used when the text involves multiple languages or is not clearly defined)
7. grc: Ancient Greek
8. enm: Middle English
9. en-CA: English (Canada)
10. ger: German
11. jpn: Japanese
12. ara: Arabic
13. nl: Dutch
14. zho: Chinese (Mandarin)
15. lat: Latin
16. por: Portuguese
17. srp: Serbian
18. ita: Italian
19. rus: Russian
20. msa: Malay (Standard Malay)
21. glg: Galician
22. wel: Welsh
23. swe: Swedish
24. nor: Norwegian
25. tur: Turkish
26. gla: Scottish Gaelic
27. ale: Aleut (a language spoken in Alaska)

* If needed, publisher issues:

Publisher: Scholastic Inc.
  - Similar: Scholastic  Inc. (Similarity: 100%)
  - Similar: Scholastic Inc (Similarity: 100%)
  - Similar: Scholastic (Similarity: 83%)
  - Similar: Hippo/Scholastic (Similarity: 80%)
Publisher: Scholastic
  - Similar: Scholastic Inc. (Similarity: 83%)
  - Similar: Scholastic  Inc. (Similarity: 83%)
  - Similar: Scholastic Inc (Similarity: 83%)
Publisher: Gramercy Books
  - Similar: Gallery Books (Similarity: 81%)
Publisher: Del Rey Books
  - Similar: Del Rey Books (NY) (Similarity: 90%)
Publisher: Random House Audio
  - Similar: Random House Anchor (Similarity: 81%)
  - Similar: Random House (Similarity: 80%)
  - Similar: Random house (Similarity: 80%)
Publisher: Ballantine Books
  - Similar: BALLANTINE BOOKS (Similarity: 100%)
  - Similar: Ballantine Books (NY) (Similarity: 91%)
  - Similar: Atlantic Books (Similarity: 80%)
  - Similar: Del Rey/Ballantine Books (Similarity: 80%)
Publisher: Houghton Mifflin Harcourt
  - Similar: Houghton Mifflin Harcourt P (Similarity: 96%)
  - Similar: Rux Martin/Houghton Mifflin Harcourt (Similarity: 82%)
Publisher: The New Press
  - Similar: New Press  The (Similarity: 100%)
  - Similar: New Press (Similarity: 82%)
Publisher: iUniverse
  - Similar: Universe (Similarity: 94%)
Publisher: Ivy Books
  - Similar: IVP Books (Similarity: 89%)
  - Similar: It Books (Similarity: 82%)
Publisher: Harper
  - Similar: HarperOne (Similarity: 80%)
  - Similar: Harpperen (Similarity: 80%)
Publisher: FT Press
  - Similar: MIT Press (Similarity: 82%)
  - Similar: Fab Press (Similarity: 82%)
Publisher: Farrar  Straus and Giroux
  - Similar: Farrar Straus and Giroux (Similarity: 100%)
  - Similar: Farrar  Straus and Giroux (NY) (Similarity: 94%)
  - Similar: Farrar  Straus and Giroux (BYR) (Similarity: 92%)
  - Similar: Farrar Straus Giroux (Similarity: 91%)
  - Similar: Farrar  Straus & Giroux (Similarity: 91%)
  - Similar: Farrar  Strauss & Giroux-3pl (Similarity: 86%)
Publisher: Farrar Straus Giroux
  - Similar: Farrar  Straus & Giroux (Similarity: 100%)
  - Similar: Farrar  Straus and Giroux (Similarity: 91%)
  - Similar: Farrar Straus and Giroux (Similarity: 91%)
  - Similar: Farrar  Strauss & Giroux-3pl (Similarity: 89%)
  - Similar: Farrar  Straus and Giroux (NY) (Similarity: 85%)
  - Similar: Farrar  Straus and Giroux (BYR) (Similarity: 83%)
Publisher: Dramatists Play Service
  - Similar: Dramatists Play Service  Inc. (Similarity: 92%)
Publisher: Chosen Books
  - Similar: Schocken Books (Similarity: 85%)
  - Similar: Icon Books (Similarity: 82%)
  - Similar: Tin House Books (Similarity: 81%)
Publisher: Workman Publishing Company
  - Similar: Zondervan Publishing Company (Similarity: 85%)
  - Similar: Wadsworth Publishing Company (Similarity: 81%)
Publisher: Ace Books
  - Similar: Ace Book (Similarity: 94%)
  - Similar: Baen Books (Similarity: 84%)
  - Similar: Basic Books (Similarity: 80%)
  - Similar: Baker Books (Similarity: 80%)
Publisher: Tor Books
  - Similar: Orb Books (Similarity: 89%)
  - Similar: It Books (Similarity: 82%)
  - Similar: Corgi Books (Similarity: 80%)
  - Similar: Totem Books (Similarity: 80%)
  - Similar: Orbis Books (Similarity: 80%)
  - Similar: Forge Books (Similarity: 80%)
  - Similar: Honor Books (Similarity: 80%)
  - Similar: Orion Books (Similarity: 80%)
  - Similar: Orbit Books (Similarity: 80%)
Publisher: Delacorte Books for Young Readers
  - Similar: Dutton Books for Young Readers (Similarity: 83%)
  - Similar: Doubleday Books for Young Readers (Similarity: 82%)
Publisher: Thomas Nelson
  - Similar: Tommy Nelson (Similarity: 80%)
Publisher: Penguin Classics
  - Similar: Penguin Classic (Similarity: 97%)
  - Similar: Penguin (Non-Classics) (Similarity: 89%)
Publisher: Penguin Books
  - Similar: Penguin Books (NY) (Similarity: 90%)
  - Similar: Penguin Books Ltd (Similarity: 87%)
  - Similar: Penguin Books (NYC) (Similarity: 87%)
  - Similar: Penguin Books Ltd. (Similarity: 87%)
  - Similar: Puffin Books (Similarity: 80%)
Publisher: Barnes & Noble Classics
  - Similar: Barnes  Noble Classics (Similarity: 100%)
Publisher: Dover Publications
  - Similar: Dover Publications (NY) (Similarity: 92%)
  - Similar: Dover Publications  Inc. (Similarity: 90%)
  - Similar: Dover Publications (NYC) (Similarity: 90%)
  - Similar: Lerner Publications (Similarity: 81%)
  - Similar: Arts Publications (Similarity: 80%)
Publisher: Barnes & Noble
  - Similar: Barnes  Noble (Similarity: 100%)
  - Similar: Barnes & Noble Books (Similarity: 80%)
Publisher: Mira Books
  - Similar: Mira books (Similarity: 100%)
  - Similar: Miramax Books (Similarity: 87%)
  - Similar: Dial Books (Similarity: 80%)
  - Similar: Gaia Books (Similarity: 80%)
Publisher: Cambridge University Press
  - Similar: Harvard University Press (Cambridge) (Similarity: 87%)
  - Similar: Columbia University Press (Similarity: 82%)
  - Similar: Howard University Press (Similarity: 82%)
  - Similar: Middlesex University Press (Similarity: 81%)
  - Similar: Cornell University Press (Similarity: 80%)
  - Similar: Harvard University Press (Similarity: 80%)
Publisher: Clarity Press
  - Similar: Conari Press (Similarity: 80%)
  - Similar: Polity Press (Similarity: 80%)
Publisher: Harper Perennial
  - Similar: HarperPerennial (Similarity: 97%)
  - Similar: Harper Perennial (NYC) (Similarity: 89%)
  - Similar: Ecco/Harper Perennial (Similarity: 86%)
  - Similar: HarperCollins Perennial (Similarity: 82%)
Publisher: William Morrow
  - Similar: William Morrow / Harper (Similarity: 80%)
Publisher: New Directions Publishing
  - Similar: New Directions Publishing Corporation (Similarity: 81%)
Publisher: Grove Press
  - Similar: Grove Press (NYC) (Similarity: 85%)
Publisher: Barnes  Noble Classics
  - Similar: Barnes & Noble Classics (Similarity: 100%)
Publisher: Pantheon
  - Similar: Panther (Similarity: 80%)
Publisher: Sterling Children's Books
  - Similar: Viking Children's Books (Similarity: 83%)
  - Similar: Simon & Schuster Childrens Books (Similarity: 80%)
Publisher: Simon & Schuster
  - Similar: Simon  Schuster (Similarity: 100%)
  - Similar: Simon Schuster (Similarity: 100%)
  - Similar: Simon and Schuster (Similarity: 88%)
  - Similar: Simon  Schuster Audio (Similarity: 82%)
  - Similar: Simon & Schuster Audio (Similarity: 82%)
Publisher: Abdo Publishing Company
  - Similar: Book Publishing Company (Similarity: 91%)
  - Similar: Book Publishing Company (TN) (Similarity: 86%)
  - Similar: Owl Publishing Company (Similarity: 80%)
Publisher: French & European
  - Similar: French & European Pubns (Similarity: 83%)
Publisher: Random House Books for Young Readers
  - Similar: Random House for Young Readers (Similarity: 91%)
  - Similar: Random House  for Young Readers (Similarity: 91%)
  - Similar: HMH Books for Young Readers (Similarity: 83%)
Publisher: Ace
  - Similar: ACE (Similarity: 100%)
Publisher: Pocket Books
  - Similar: Pocket Star Books (Similarity: 83%)
Publisher: Samuel French  Inc.
  - Similar: Samuel French Ltd (Similarity: 82%)
Publisher: Princeton University Press
  - Similar: Princeton University Press (NJ) (Similarity: 95%)
  - Similar: Bollingen/Princeton University Press (NJ) (Similarity: 80%)
Publisher: W. W. Norton & Company
  - Similar: W. W. Norton  Company (Similarity: 100%)
  - Similar: W.W. Norton & Company (Similarity: 100%)
  - Similar: W.W. Norton  Company (Similarity: 100%)
  - Similar: W.W. Norton & Company (NY) (Similarity: 92%)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 90%)
  - Similar: W. W. Norton and Company (Similarity: 90%)
  - Similar: W. W. Norton & Co. (Similarity: 84%)
  - Similar: W.W. Norton & Company  Inc. (NY) (Similarity: 84%)
Publisher: HMH Books for Young Readers
  - Similar: Knopf Books for Young Readers (Similarity: 86%)
  - Similar: Random House Books for Young Readers (Similarity: 83%)
  - Similar: Dutton Books for Young Readers (Similarity: 81%)
  - Similar: Viking Books for Young Readers (Similarity: 81%)
Publisher: Penguin
  - Similar: Penguin UK (Similarity: 82%)
Publisher: Harper Perennial Modern Classics
  - Similar: Harper Perennial Classics (Similarity: 88%)
  - Similar: HarperPerennial / Perennial Classics (Similarity: 85%)
Publisher: Knopf Publishing Group
  - Similar: Orion Publishing Group (Similarity: 82%)
  - Similar: B&H Publishing Group (Similarity: 81%)
  - Similar: Knopf Doubleday Publishing Group (Similarity: 81%)
  - Similar: W Publishing Group (Similarity: 80%)
Publisher: Everyman's Library
  - Similar: Everymans Library (Similarity: 91%)
  - Similar: Everyman's Library 234 (Similarity: 90%)
  - Similar: Everyman's Library Classics (Similarity: 80%)
Publisher: Sun and Moon Press
  - Similar: Sun & Moon Press (Similarity: 88%)
Publisher: Henry Holt & Company
  - Similar: Henry Holt and Company (Similarity: 90%)
Publisher: Penguin Books Ltd. (London)
  - Similar: Penguin Books (London) (Similarity: 91%)
  - Similar: Penguin Books Ltd (Similarity: 83%)
  - Similar: Penguin Books Ltd. (Similarity: 83%)
Publisher: Simon  Schuster
  - Similar: Simon & Schuster (Similarity: 100%)
  - Similar: Simon Schuster (Similarity: 100%)
  - Similar: Simon and Schuster (Similarity: 88%)
  - Similar: Simon  Schuster Audio (Similarity: 82%)
  - Similar: Simon & Schuster Audio (Similarity: 82%)
Publisher: W. W. Norton  Company
  - Similar: W. W. Norton & Company (Similarity: 100%)
  - Similar: W.W. Norton & Company (Similarity: 100%)
  - Similar: W.W. Norton  Company (Similarity: 100%)
  - Similar: W.W. Norton & Company (NY) (Similarity: 92%)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 90%)
  - Similar: W. W. Norton and Company (Similarity: 90%)
  - Similar: W. W. Norton & Co. (Similarity: 84%)
  - Similar: W.W. Norton & Company  Inc. (NY) (Similarity: 84%)
Publisher: Lerner Publications
  - Similar: Dover Publications (Similarity: 81%)
  - Similar: Lectorum Publications (Similarity: 80%)
  - Similar: Dover Publications (NY) (Similarity: 80%)
Publisher: Listening Library (Audio)
  - Similar: Listening Library (Similarity: 85%)
Publisher: Random House
  - Similar: Random house (Similarity: 100%)
  - Similar: Random House (NY) (Similarity: 89%)
  - Similar: Random House  Inc. (Similarity: 86%)
  - Similar: Random House Inc. (Similarity: 86%)
  - Similar: Random House Audio (Similarity: 80%)
  - Similar: Random House Trade (Similarity: 80%)
Publisher: HarperCollins Publishers
  - Similar: HarperCollinsPublishers (Similarity: 98%)
  - Similar: HarperCollins Publishers  Inc. (Similarity: 92%)
  - Similar: HarperCollins Publishers Ltd (Similarity: 92%)
  - Similar: Collins Publishers (Similarity: 86%)
  - Similar: Atlas Books/HarperCollins Publishers (Similarity: 80%)
Publisher: HarperTorch
  - Similar: Hartorch (Similarity: 84%)
Publisher: Rockport Publishers
  - Similar: Worth Publishers (Similarity: 80%)
Publisher: Signet Classics
  - Similar: Signet Classic (Similarity: 97%)
  - Similar: Scribner Classics (Similarity: 81%)
Publisher: Oxford University Press
  - Similar: Oxford University Press  USA (Similarity: 92%)
  - Similar: Howard University Press (Similarity: 87%)
  - Similar: University Press of Florida (Similarity: 84%)
  - Similar: Cornell University Press (Similarity: 81%)
  - Similar: Harvard University Press (Similarity: 81%)
  - Similar: University of Utah Press (Similarity: 81%)
  - Similar: Duke University Press Books (Similarity: 80%)
Publisher: Plume
  - Similar: Lumen (Similarity: 80%)
Publisher: Signet Book
  - Similar: Signet Books (NY) (Similarity: 85%)
Publisher: Dutton
  - Similar: Sutton (Similarity: 83%)
Publisher: Lyons Press
  - Similar: Oni Press (Similarity: 80%)
  - Similar: LSU Press (Similarity: 80%)
Publisher: Alliance Publishing
  - Similar: Kaplan Publishing (Similarity: 83%)
  - Similar: Planeta Publishing (Similarity: 81%)
  - Similar: Pelican Publishing (Similarity: 81%)
Publisher: Alfred A. Knopf
  - Similar: Alfred A.Knopf (Similarity: 100%)
  - Similar: Alfred A. Knopf  Inc. (Similarity: 88%)
Publisher: Oxford University Press  USA
  - Similar: Oxford University Press (Similarity: 92%)
  - Similar: University of Utah Press (Similarity: 86%)
  - Similar: Howard University Press (Similarity: 80%)
Publisher: Plaza y Janes
  - Similar: Plaza y Janés (Similarity: 96%)
  - Similar: Plaza & Janés (Similarity: 87%)
Publisher: Vintage Books USA
  - Similar: Vintage Books (Similarity: 87%)
Publisher: Wisdom Publications
  - Similar: Simon Publications (Similarity: 81%)
Publisher: Geoplaneta
  - Similar: Planeta (Similarity: 82%)
Publisher: Mariner Books
  - Similar: Warner Books (Similarity: 88%)
  - Similar: Gardners Books (Similarity: 81%)
  - Similar: TIme Warner Books (Similarity: 80%)
Publisher: HarperCollins Perennial
  - Similar: Harper Perennial (HarperCollins) (Similarity: 87%)
  - Similar: Harper Perennial (Similarity: 82%)
Publisher: Bantam Books
  - Similar: Bantam Books (NY) (Similarity: 89%)
  - Similar: Bantam Books  Inc. (Similarity: 86%)
  - Similar: Baen Books (Similarity: 82%)
  - Similar: Baronet Books (Similarity: 80%)
Publisher: Basic Books
  - Similar: Basic Books (AZ) (Similarity: 88%)
  - Similar: Ace Books (Similarity: 80%)
  - Similar: BBC Books (Similarity: 80%)
Publisher: HarperOne
  - Similar: Harper (Similarity: 80%)
Publisher: HarperCollins
  - Similar: Harpercollins (Similarity: 100%)
  - Similar: HarperCollins UK (Similarity: 90%)
  - Similar: Ecco / HarperCollins (Similarity: 84%)
  - Similar: Avon HarperCollins (Similarity: 84%)
Publisher: VIZ Media LLC
  - Similar: VIZ Media (Similarity: 82%)
  - Similar: Viz Media (Similarity: 82%)
Publisher: It Books
  - Similar: Titan Books (Similarity: 84%)
  - Similar: Orbit Books (Similarity: 84%)
  - Similar: Ivy Books (Similarity: 82%)
  - Similar: Tor Books (Similarity: 82%)
  - Similar: MTV Books (Similarity: 82%)
  - Similar: IVP Books (Similarity: 82%)
Publisher: Random House Large Print Publishing
  - Similar: Random House Large Print (Similarity: 81%)
Publisher: Oni Press
  - Similar: Conari Press (Similarity: 86%)
  - Similar: Council Press (Similarity: 82%)
  - Similar: Phoenix Press (Similarity: 82%)
  - Similar: Odonian Press (Similarity: 82%)
  - Similar: Pi Press (Similarity: 82%)
  - Similar: Omnibus Press (Similarity: 82%)
  - Similar: Lyons Press (Similarity: 80%)
  - Similar: Naiad Press (Similarity: 80%)
Publisher: Moody Publishers
  - Similar: Moody Publishers (Chicago) (Similarity: 80%)
Publisher: Corgi Books
  - Similar: Courage Books (Similarity: 83%)
  - Similar: Orbis Books (Similarity: 82%)
  - Similar: Forge Books (Similarity: 82%)
  - Similar: Orion Books (Similarity: 82%)
  - Similar: Orbit Books (Similarity: 82%)
  - Similar: Tor Books (Similarity: 80%)
  - Similar: Orb Books (Similarity: 80%)
Publisher: Bantam Press
  - Similar: Bantem Press (Similarity: 92%)
Publisher: Gallery Books
  - Similar: Gramercy Books (Similarity: 81%)
  - Similar: Walker Books (Similarity: 80%)
Publisher: Tarcherperigee
  - Similar: TarcherPerigee (Similarity: 100%)
Publisher: Firebrand Books
  - Similar: Firebird Books (Similarity: 90%)
Publisher: Berkley Books
  - Similar: Berkley Books (NY) (Similarity: 90%)
Publisher: Recorded Books  Inc.
  - Similar: Recorded Books (Similarity: 88%)
Publisher: University of Chicago Press
  - Similar: University Of Chicago Press (Similarity: 100%)
  - Similar: University of Michigan Press (Similarity: 87%)
  - Similar: University Press of America (Similarity: 85%)
  - Similar: University of Calgary Press (Similarity: 85%)
  - Similar: University of California Press (Similarity: 81%)
  - Similar: University of Georgia Press (Similarity: 81%)
  - Similar: University Press of Florida (Similarity: 81%)
  - Similar: University of New Mexico Press (Similarity: 81%)
  - Similar: University of Arizona Press (Similarity: 81%)
  - Similar: University of Illinois Press (Similarity: 80%)
Publisher: Howard Publishing Co
  - Similar: Orion Publishing Co. (Similarity: 82%)
  - Similar: Orion Publishing Co (Similarity: 82%)
Publisher: HarperCollins Publishers  Inc.
  - Similar: HarperCollins Publishers (Similarity: 92%)
  - Similar: HarperCollinsPublishers (Similarity: 90%)
  - Similar: HarperCollins Publishers Ltd (Similarity: 89%)
Publisher: Rodale Books
  - Similar: Dial Books (Similarity: 82%)
  - Similar: Profile Books (Similarity: 80%)
Publisher: John Wiley & Sons
  - Similar: John Wiley (Similarity: 80%)
Publisher: B Books
  - Similar: BBC Books (Similarity: 88%)
  - Similar: Baen Books (Similarity: 82%)
Publisher: Clarkson Potter Publishers
  - Similar: Clarkson Potter/Publishers (Similarity: 100%)
Publisher: Putnam Publishing Group
  - Similar: W Publishing Group (Similarity: 83%)
Publisher: Prentice Hall
  - Similar: Prentice Hall PTR (Similarity: 87%)
  - Similar: Prentice Hall Press (Similarity: 81%)
Publisher: Little  Brown
  - Similar: Little Brown (Similarity: 100%)
Publisher: Harper Paperbacks
  - Similar: HarperPaperbacks (Similarity: 97%)
  - Similar: Hyperion Paperbacks (Similarity: 83%)
Publisher: Prentice Hall Press
  - Similar: Prentice Hall PTR (Similarity: 89%)
  - Similar: Prentice Hall (Similarity: 81%)
Publisher: Crown Publishing Group
  - Similar: Crown Publishing Group (NY) (Similarity: 94%)
  - Similar: Carlton Publishing Group (Similarity: 91%)
  - Similar: Broadway/Crown Publishing Group (Similarity: 83%)
  - Similar: Avery Publishing Group (Similarity: 82%)
  - Similar: B&H Publishing Group (Similarity: 81%)
  - Similar: W Publishing Group (Similarity: 80%)
Publisher: BradyGames
  - Similar: Bradygames (Similarity: 100%)
Publisher: Tor Classics
  - Similar: NYRB Classics (Similarity: 80%)
Publisher: Warner Books
  - Similar: Warner Books (NY) (Similarity: 89%)
  - Similar: Mariner Books (Similarity: 88%)
  - Similar: Gardners Books (Similarity: 85%)
  - Similar: Walker Books (Similarity: 83%)
  - Similar: TIme Warner Books (Similarity: 83%)
  - Similar: Wenner Books (Similarity: 83%)
Publisher: St. Augustines Press
  - Similar: St. Martins Press (Similarity: 80%)
Publisher: Hackett Publishing Company  Inc.
  - Similar: Hackett Publishing Company  Inc. (USA) (Similarity: 94%)
  - Similar: Hackett Publishing Company (Similarity: 93%)
  - Similar: Hackett Publishing Company (Indianapolis  IN) (Similarity: 81%)
Publisher: Cornell University Press
  - Similar: Columbia University Press (Similarity: 82%)
  - Similar: Oxford University Press (Similarity: 81%)
  - Similar: Howard University Press (Similarity: 81%)
  - Similar: Cambridge University Press (Similarity: 80%)
Publisher: W.W. Norton & Company
  - Similar: W. W. Norton & Company (Similarity: 100%)
  - Similar: W. W. Norton  Company (Similarity: 100%)
  - Similar: W.W. Norton  Company (Similarity: 100%)
  - Similar: W.W. Norton & Company (NY) (Similarity: 92%)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 90%)
  - Similar: W. W. Norton and Company (Similarity: 90%)
  - Similar: W. W. Norton & Co. (Similarity: 84%)
  - Similar: W.W. Norton & Company  Inc. (NY) (Similarity: 84%)
Publisher: Everyman's Library 234
  - Similar: Everyman's Library (Similarity: 90%)
  - Similar: Everymans Library (Similarity: 82%)
Publisher: Harvard University Press
  - Similar: Howard University Press (Similarity: 89%)
  - Similar: Harvard University Press (Cambridge) (Similarity: 83%)
  - Similar: Oxford University Press (Similarity: 81%)
  - Similar: Cambridge University Press (Similarity: 80%)
Publisher: Viking
  - Similar: Viking UK (Similarity: 80%)
Publisher: Bantam Classics
  - Similar: Bantam Classic (Similarity: 97%)
Publisher: Loeb Classical Library
  - Similar: Loeb Classical Library 307 (Similarity: 92%)
  - Similar: Loeb Classical Library 443 (Similarity: 92%)
  - Similar: Modern Library Classics (Similarity: 80%)
Publisher: New American Library
  - Similar: Library of America (Similarity: 84%)
  - Similar: Library of America (NY) (Similarity: 83%)
Publisher: Signet Books (NY)
  - Similar: Signet Book (Similarity: 85%)
Publisher: Little Brown and Company
  - Similar: Little  Brown and Company (Similarity: 100%)
  - Similar: Little  Brown & Company (Similarity: 91%)
Publisher: Simon Schuster
  - Similar: Simon & Schuster (Similarity: 100%)
  - Similar: Simon  Schuster (Similarity: 100%)
  - Similar: Simon and Schuster (Similarity: 88%)
  - Similar: Simon  Schuster Audio (Similarity: 82%)
  - Similar: Simon & Schuster Audio (Similarity: 82%)
Publisher: Granta UK
  - Similar: Granta (Similarity: 80%)
Publisher: Longman Publishing Group
  - Similar: Orion Publishing Group (Similarity: 83%)
  - Similar: Orion Publishing Group  Ltd. (Similarity: 80%)
Publisher: University of California Press
  - Similar: University Press of Florida (Similarity: 88%)
  - Similar: University of Arizona Press (Similarity: 88%)
  - Similar: University of Georgia Press (Similarity: 84%)
  - Similar: University of Calgary Press (Similarity: 84%)
  - Similar: University of Illinois Press (Similarity: 83%)
  - Similar: University of Arkansas Press (Similarity: 83%)
  - Similar: University Press of Kansas (Similarity: 82%)
  - Similar: University of Alaska Press (Similarity: 82%)
  - Similar: University of Chicago Press (Similarity: 81%)
  - Similar: University Of Chicago Press (Similarity: 81%)
  - Similar: University of South Carolina Press (Similarity: 81%)
  - Similar: University of North Carolina Press (Similarity: 81%)
Publisher: Ecco Press
  - Similar: ECW Press (Similarity: 84%)
  - Similar: CRC Press (Similarity: 84%)
  - Similar: Beacon Press (Similarity: 82%)
Publisher: Wayne State University Press
  - Similar: Kent State University Press (Similarity: 80%)
  - Similar: Ohio State University Press (Similarity: 80%)
Publisher: Three Rivers Press (CA)
  - Similar: Three Rivers Press (Similarity: 92%)
Publisher: W.W. Norton
  - Similar: W. W. Norton & Co. (Similarity: 87%)
Publisher: Warner Books (NY)
  - Similar: Warner Books (Similarity: 89%)
  - Similar: TIme Warner Books (Similarity: 81%)
Publisher: Alfred A. Knopf Books for Young Readers
  - Similar: Knopf Books for Young Readers (Similarity: 87%)
Publisher: Pantheon Books (NY)
  - Similar: Pantheon Books (Similarity: 90%)
Publisher: Penguin Books Ltd
  - Similar: Penguin Books Ltd. (Similarity: 100%)
  - Similar: Penguin Books Limited (Similarity: 89%)
  - Similar: Penguin Books (Similarity: 87%)
  - Similar: Penguin Book Limited (Similarity: 86%)
  - Similar: Penguin Books (London) (Similarity: 86%)
  - Similar: Penguin Books (NY) (Similarity: 85%)
  - Similar: Plume (Penguin Books Ltd) (Similarity: 85%)
  - Similar: Penguin Books Ltd. (London) (Similarity: 83%)
  - Similar: Penguin Books  Limited (UK) (Similarity: 83%)
  - Similar: Penguin Books (NYC) (Similarity: 82%)
  - Similar: Penguin Books Canada (Similarity: 81%)
Publisher: Modern Library
  - Similar: Modern Library (NY) (Similarity: 90%)
  - Similar: The Modern Library (Similarity: 88%)
Publisher: Farrar  Straus and Giroux (NY)
  - Similar: Farrar  Straus and Giroux (Similarity: 94%)
  - Similar: Farrar Straus and Giroux (Similarity: 94%)
  - Similar: Farrar  Straus and Giroux (BYR) (Similarity: 87%)
  - Similar: Farrar Straus Giroux (Similarity: 85%)
  - Similar: Farrar  Straus & Giroux (Similarity: 85%)
  - Similar: Farrar  Strauss & Giroux-3pl (Similarity: 81%)
Publisher: Planeta Publishing
  - Similar: Pan Publishing (Similarity: 88%)
  - Similar: Kaplan Publishing (Similarity: 86%)
  - Similar: Pelican Publishing (Similarity: 83%)
  - Similar: Delta Publishing (Similarity: 82%)
  - Similar: Alliance Publishing (Similarity: 81%)
  - Similar: AVA Publishing (Similarity: 81%)
  - Similar: Plexus Publishing (Similarity: 80%)
Publisher: Scholastic Paperbacks
  - Similar: Apple Paperbacks (Scholastic) (Similarity: 88%)
Publisher: Library of America
  - Similar: Library of America (NY) (Similarity: 92%)
  - Similar: New American Library (Similarity: 84%)
  - Similar: Library of American Comics (Similarity: 82%)
Publisher: Collector's Library
  - Similar: Collectors Library (Similarity: 92%)
Publisher: Spark Publishing
  - Similar: P & R Publishing (Similarity: 80%)
Publisher: Arthur A. Levine
  - Similar: Arthur A. Levine Books (Similarity: 83%)
Publisher: Bloomsbury Publishing
  - Similar: Bloomsbury Publishing PLC (Similarity: 91%)
  - Similar: Albury Publishing (Similarity: 84%)
Publisher: Plume Books
  - Similar: Philomel Books (Similarity: 80%)
Publisher: Viking Books
  - Similar: Inkling Books (Similarity: 88%)
  - Similar: Living Books (Similarity: 83%)
  - Similar: Vintage Books (Similarity: 80%)
Publisher: MIT Press
  - Similar: MIT Press (MA) (Similarity: 86%)
  - Similar: FT Press (Similarity: 82%)
  - Similar: Amistad Press (Similarity: 82%)
  - Similar: Pi Press (Similarity: 82%)
  - Similar: The MIT Press (Similarity: 82%)
Publisher: Little  Brown and Company
  - Similar: Little Brown and Company (Similarity: 100%)
  - Similar: Little  Brown & Company (Similarity: 91%)
Publisher: MTV Books
  - Similar: It Books (Similarity: 82%)
Publisher: G.P. Putnam's Sons
  - Similar: G. P. Putnam's Sons (Similarity: 100%)
Publisher: Jones Books
  - Similar: Jove Books (Similarity: 86%)
Publisher: Simon  Schuster Audio
  - Similar: Simon & Schuster Audio (Similarity: 100%)
  - Similar: Simon and Schuster (Similarity: 89%)
  - Similar: Simon & Schuster (Similarity: 82%)
  - Similar: Simon  Schuster (Similarity: 82%)
  - Similar: Simon Schuster (Similarity: 82%)
Publisher: Penguin Books (Penguin Classics)
  - Similar: Penguin Books /Penguin Classics (Similarity: 100%)
  - Similar: Penguin Books/Penguin Popular Classics (Similarity: 88%)
Publisher: Simon & Schuster Paperbacks
  - Similar: Simon & Schuster Aladddin Paperbacks (Similarity: 85%)
Publisher: Berkley Publishing Group
  - Similar: The Berkley Publishing Group (Similarity: 92%)
  - Similar: Avery Publishing Group (Similarity: 87%)
  - Similar: Berkley Publishing (Similarity: 86%)
  - Similar: B&H Publishing Group (Similarity: 82%)
Publisher: Rutgers University Press
  - Similar: University of Texas Press (Similarity: 82%)
  - Similar: Summit University Press (Similarity: 81%)
  - Similar: Texas A&M University Press (Similarity: 80%)
Publisher: Three Rivers Press
  - Similar: Three Rivers Press (CA) (Similarity: 92%)
Publisher: Houghton Mifflin Company
  - Similar: Houghton Mifflin (Similarity: 80%)
Publisher: Peter Smith Publisher
  - Similar: Peter Smith Publisher  Inc. (Similarity: 91%)
Publisher: Naxos Audiobooks
  - Similar: Nova Audio Books (Similarity: 81%)
Publisher: Candlewick Press
  - Similar: Candlewick Press (MA) (Similarity: 91%)
  - Similar: Academic Press (Similarity: 80%)
Publisher: Penguin Audio
  - Similar: Penguin Audio UK (Similarity: 90%)
Publisher: University of Illinois Press
  - Similar: University of California Press (Similarity: 83%)
  - Similar: University of Michigan Press (Similarity: 82%)
  - Similar: University Press of Kansas (Similarity: 81%)
  - Similar: University of Minnesota Press (Similarity: 81%)
  - Similar: University Of Minnesota Press (Similarity: 81%)
  - Similar: University of Alaska Press (Similarity: 81%)
  - Similar: University of Chicago Press (Similarity: 80%)
  - Similar: University Of Chicago Press (Similarity: 80%)
  - Similar: University of Arizona Press (Similarity: 80%)
Publisher: University Press of Kansas
  - Similar: University of Arkansas Press (Similarity: 96%)
  - Similar: University of Arizona Press (Similarity: 87%)
  - Similar: University of Alaska Press (Similarity: 85%)
  - Similar: University Press of America (Similarity: 83%)
  - Similar: University of Calgary Press (Similarity: 83%)
  - Similar: University of California Press (Similarity: 82%)
  - Similar: University of Illinois Press (Similarity: 81%)
  - Similar: University of Michigan Press (Similarity: 81%)
  - Similar: University of Massachusetts Press (Similarity: 81%)
  - Similar: University Press of Kentucky (Similarity: 81%)
  - Similar: University of Nebraska Press (Similarity: 81%)
  - Similar: Indiana University Press (Similarity: 80%)
Publisher: Jove Books
  - Similar: Jones Books (Similarity: 86%)
Publisher: Orion Publishing
  - Similar: Orion Publishing Co. (Similarity: 91%)
  - Similar: Orion Publishing Co (Similarity: 91%)
  - Similar: Merlin Publishing (Similarity: 85%)
  - Similar: Modern Publishing (Similarity: 85%)
  - Similar: Orion Publishing Group (Similarity: 84%)
  - Similar: Ig Publishing (Similarity: 83%)
  - Similar: Pan Publishing (Similarity: 80%)
  - Similar: IDW Publishing (Similarity: 80%)
Publisher: Alfred A.Knopf
  - Similar: Alfred A. Knopf (Similarity: 100%)
  - Similar: Alfred A. Knopf  Inc. (Similarity: 88%)
Publisher: Schirmer Mosel
  - Similar: Schirmer/Mosel (Similarity: 100%)
Publisher: Hachette Books
  - Similar: Fawcett Books (Similarity: 81%)
Publisher: North Atlantic Books
  - Similar: Atlantic Books (Similarity: 82%)
  - Similar: Atlantic Books (UK) (Similarity: 81%)
Publisher: Barrons Educational Series
  - Similar: Barron's Educational Series (Similarity: 94%)
Publisher: Houghton Mifflin
  - Similar: Houghton Mifflin Company (Similarity: 80%)
Publisher: Simon  Schuster Books for Young Readers
  - Similar: Simon & Schuster Books for Young Readers (Similarity: 100%)
Publisher: Yale University Press
  - Similar: Wesleyan University Press (Similarity: 83%)
  - Similar: University of Utah Press (Similarity: 80%)
Publisher: Bloomsbury Publishing PLC
  - Similar: Bloomsbury Publishing (Similarity: 91%)
Publisher: Health Communications Inc
  - Similar: Health Communications (Similarity: 91%)
  - Similar: Hearst Communications (Similarity: 83%)
Publisher: Owl Publishing Company
  - Similar: Dell Publishing Company (Similarity: 89%)
  - Similar: Pelican Publishing Company (Similarity: 83%)
  - Similar: Abdo Publishing Company (Similarity: 80%)
  - Similar: MacMillan Publishing Company (Similarity: 80%)
  - Similar: Book Publishing Company (Similarity: 80%)
  - Similar: Crossroad Publishing Company (Similarity: 80%)
Publisher: Thames  Hudson
  - Similar: Thames & Hudson (Similarity: 100%)
  - Similar: Thames & Hudson Ltd (Similarity: 87%)
Publisher: Duke University Press Books
  - Similar: Oxford University Press (Similarity: 80%)
Publisher: Crown Publishing Group (NY)
  - Similar: Crown Publishing Group (Similarity: 94%)
  - Similar: Carlton Publishing Group (Similarity: 86%)
  - Similar: B&H Publishing Group (Similarity: 80%)
Publisher: Shaw Books
  - Similar: Daw Books (Similarity: 84%)
  - Similar: DAW Books (Similarity: 84%)
  - Similar: Seal Books (Similarity: 80%)
Publisher: Roxford Books
  - Similar: Burford Books (Similarity: 85%)
Publisher: HarperTeen
  - Similar: Harper Teen (Similarity: 95%)
  - Similar: Harpperen (Similarity: 84%)
Publisher: Clarion Books
  - Similar: Orion Books (Similarity: 83%)
  - Similar: Creation Books (Similarity: 81%)
  - Similar: Nation Books (Similarity: 80%)
Publisher: Arcade Publishing
  - Similar: Arcadia Publishing (Similarity: 91%)
  - Similar: AVA Publishing (Similarity: 84%)
  - Similar: Arcadia Publishing (SC) (Similarity: 84%)
  - Similar: Arbordale Publishing (Similarity: 81%)
  - Similar: Dell Publishing (Similarity: 81%)
  - Similar: DK Publishing (Similarity: 80%)
Publisher: Harlequin Romance
  - Similar: Harlequin Superromance (Similarity: 87%)
Publisher: Council Press
  - Similar: Oni Press (Similarity: 82%)
  - Similar: Conari Press (Similarity: 80%)
Publisher: The Modern Library
  - Similar: Modern Library (Similarity: 88%)
  - Similar: Modern Library (NY) (Similarity: 86%)
  - Similar: The Modern Library New York (Similarity: 80%)
Publisher: David C Cook
  - Similar: David C. Cook (Similarity: 100%)
Publisher: Basic Books (AZ)
  - Similar: Basic Books (Similarity: 88%)
Publisher: Encounter Books
  - Similar: Wenner Books (Similarity: 81%)
Publisher: Walker Books Ltd
  - Similar: Walker Books (Similarity: 86%)
Publisher: Farrar Straus and Giroux
  - Similar: Farrar  Straus and Giroux (Similarity: 100%)
  - Similar: Farrar  Straus and Giroux (NY) (Similarity: 94%)
  - Similar: Farrar  Straus and Giroux (BYR) (Similarity: 92%)
  - Similar: Farrar Straus Giroux (Similarity: 91%)
  - Similar: Farrar  Straus & Giroux (Similarity: 91%)
  - Similar: Farrar  Strauss & Giroux-3pl (Similarity: 86%)
Publisher: Scholastic Books
  - Similar: Scholastic Audio Books (Similarity: 84%)
  - Similar: Classic Books (Similarity: 83%)
Publisher: The Harvill Press
  - Similar: Harvill Press (Similarity: 87%)
Publisher: Simon and Schuster
  - Similar: Simon  Schuster Audio (Similarity: 89%)
  - Similar: Simon & Schuster Audio (Similarity: 89%)
  - Similar: Simon & Schuster (Similarity: 88%)
  - Similar: Simon  Schuster (Similarity: 88%)
  - Similar: Simon Schuster (Similarity: 88%)
Publisher: Livre de Poche
  - Similar: Le Livre de Poche (Similarity: 90%)
Publisher: Loeb Classical Library 307
  - Similar: Loeb Classical Library (Similarity: 92%)
  - Similar: Loeb Classical Library 443 (Similarity: 92%)
Publisher: Hackett Publishing Company  Inc. (USA)
  - Similar: Hackett Publishing Company  Inc. (Similarity: 94%)
  - Similar: Hackett Publishing Company (Similarity: 87%)
Publisher: University Of Chicago Press
  - Similar: University of Chicago Press (Similarity: 100%)
  - Similar: University of Michigan Press (Similarity: 87%)
  - Similar: University Press of America (Similarity: 85%)
  - Similar: University of Calgary Press (Similarity: 85%)
  - Similar: University of California Press (Similarity: 81%)
  - Similar: University of Georgia Press (Similarity: 81%)
  - Similar: University Press of Florida (Similarity: 81%)
  - Similar: University of New Mexico Press (Similarity: 81%)
  - Similar: University of Arizona Press (Similarity: 81%)
  - Similar: University of Illinois Press (Similarity: 80%)
Publisher: Totem Books
  - Similar: Tor Books (Similarity: 80%)
Publisher: HarperCollins Espanol
  - Similar: Ecco / HarperCollins (Similarity: 82%)
  - Similar: Avon HarperCollins (Similarity: 82%)
Publisher: Faber  Faber
  - Similar: Faber & Faber (Similarity: 100%)
  - Similar: Faber and Faber (Similarity: 85%)
  - Similar: Faber & Faber Ltd. (Similarity: 85%)
Publisher: Hodder Children's Books
  - Similar: Orion Children's Books (Similarity: 84%)
  - Similar: Harcourt Children's Books (Similarity: 83%)
Publisher: Random House Trade
  - Similar: Random House Vintage (Similarity: 84%)
  - Similar: Random House (Vintage) (Similarity: 84%)
  - Similar: Random House (Similarity: 80%)
  - Similar: Random house (Similarity: 80%)
Publisher: New Riders Publishing
  - Similar: New World Publishing (Similarity: 83%)
Publisher: Avon Books
  - Similar: Avon Books (P) (Similarity: 91%)
  - Similar: Avenel Books (Similarity: 82%)
  - Similar: Alyson Books (Similarity: 82%)
  - Similar: Baen Books (Similarity: 80%)
Publisher: William Morrow & Company (NYC)
  - Similar: William Morrow & Company (Similarity: 92%)
  - Similar: William Morrow & Company  Inc. (Similarity: 85%)
Publisher: Hyperion
  - Similar: Hyperio (Similarity: 93%)
  - Similar: Hiperión (Similarity: 80%)
Publisher: Broadway/Crown Publishing Group
  - Similar: Crown Publishing Group (Similarity: 83%)
Publisher: Roca Editorial
  - Similar: Editorial RM (Similarity: 85%)
  - Similar: Norma Editorial (Similarity: 83%)
Publisher: St. Martin's Press
  - Similar: St. Martins Press (Similarity: 91%)
  - Similar: St. Martins Press-3PL (Similarity: 81%)
Publisher: Pocket Star Books
  - Similar: Pocket Books/Star Trek (Similarity: 87%)
  - Similar: Pocket Books (Similarity: 83%)
Publisher: Bethany House Publishers
  - Similar: Arkham House Publishers (Similarity: 81%)
Publisher: Harlequin Superromance
  - Similar: Harlequin Romance (Similarity: 87%)
Publisher: Merlin Publishing
  - Similar: Modern Publishing (Similarity: 88%)
  - Similar: Pelican Publishing (Similarity: 86%)
  - Similar: Orion Publishing (Similarity: 85%)
  - Similar: Meisha Merlin Publishing (Similarity: 83%)
  - Similar: Dell Publishing (Similarity: 81%)
  - Similar: Methuen Publishing (Similarity: 80%)
  - Similar: Berkley Publishing (Similarity: 80%)
  - Similar: Ig Publishing (Similarity: 80%)
Publisher: International Publishers
  - Similar: International Publishers (NYC) (Similarity: 92%)
Publisher: Dell Laurel-Leaf
  - Similar: Laurel Leaf (Similarity: 81%)
Publisher: Seal Books
  - Similar: Regal Books (Similarity: 86%)
  - Similar: NAL Books (Similarity: 84%)
  - Similar: Shaw Books (Similarity: 80%)
  - Similar: Dial Books (Similarity: 80%)
Publisher: Atlantic Monthly Press
  - Similar: Atlantic Monthly Press (NYC) (Similarity: 92%)
Publisher: Simon & Schuster Audio
  - Similar: Simon  Schuster Audio (Similarity: 100%)
  - Similar: Simon and Schuster (Similarity: 89%)
  - Similar: Simon & Schuster (Similarity: 82%)
  - Similar: Simon  Schuster (Similarity: 82%)
  - Similar: Simon Schuster (Similarity: 82%)
Publisher: Granta
  - Similar: Granta UK (Similarity: 80%)
Publisher: Penguin Classics Deluxe Editions
  - Similar: Penguin Classics Deluxe Edition (Similarity: 98%)
Publisher: Orion Children's Books
  - Similar: Hodder Children's Books (Similarity: 84%)
  - Similar: Egmont Children's Books (Similarity: 84%)
  - Similar: Dutton Children's Books (Similarity: 84%)
  - Similar: Hyperion Books for Children (Similarity: 82%)
  - Similar: Harcourt Children's Books (Similarity: 81%)
Publisher: MacMillan UK
  - Similar: Macmillan (Similarity: 86%)
  - Similar: MacMillan (Similarity: 86%)
  - Similar: Pan MacMillan (Similarity: 80%)
Publisher: Macmillan
  - Similar: MacMillan (Similarity: 100%)
  - Similar: MacMillan UK (Similarity: 86%)
  - Similar: Pan MacMillan (Similarity: 82%)
Publisher: William Morrow & Company
  - Similar: William Morrow & Company (NYC) (Similarity: 92%)
  - Similar: William Morrow & Company  Inc. (Similarity: 92%)
Publisher: Fawcett Books
  - Similar: Hachette Books (Similarity: 81%)
  - Similar: Fawcett Crest Books (Similarity: 81%)
Publisher: Gardners Books
  - Similar: Warner Books (Similarity: 85%)
  - Similar: Mariner Books (Similarity: 81%)
Publisher: Harcourt Brace Jovanovich
  - Similar: Harcourt  Brace  Jovanovich (Similarity: 100%)
  - Similar: Harcourt Brace Jovanovich (NY) (Similarity: 94%)
  - Similar: Harcourt  Brace Jovanovich/Harvest (Similarity: 86%)
Publisher: Ellora's Cave
  - Similar: Elloras Cave (Similarity: 96%)
Publisher: Counterpoint
  - Similar: Counterpoint LLC (Similarity: 86%)
  - Similar: Center Point (Similarity: 83%)
Publisher: Regnery Publishing
  - Similar: Rosenberg Publishing (Similarity: 84%)
  - Similar: Virgin Publishing (Similarity: 80%)
  - Similar: Revive Publishing (Similarity: 80%)
Publisher: Bloomsbury
  - Similar: Bloomsbury UK (Similarity: 87%)
  - Similar: Bloomsbury USA (Similarity: 83%)
  - Similar: Bloomsbury (NYC) (Similarity: 83%)
Publisher: National Geographic Society
  - Similar: National Geographic (Similarity: 83%)
Publisher: Harper Voyager
  - Similar: HarperVoyager (Similarity: 96%)
Publisher: Bantam Spectra
  - Similar: BantamSpectra (Similarity: 96%)
  - Similar: Bantam Spectra (NY) (Similarity: 90%)
Publisher: Beacon Press
  - Similar: Ecco Press (Similarity: 82%)
Publisher: Plexus Publishing Ltd
  - Similar: Plexus Publishing (Similarity: 89%)
Publisher: Simon & Schuster Childrens Books
  - Similar: Sterling Children's Books (Similarity: 80%)
Publisher: IVP Books
  - Similar: Ivy Books (Similarity: 89%)
  - Similar: It Books (Similarity: 82%)
Publisher: Random House Large Print
  - Similar: Random House Large Print Publishing (Similarity: 81%)
Publisher: Ronin Publishing (CA)
  - Similar: Green Ronin Publishing (Similarity: 83%)
Publisher: Bloomsbury UK
  - Similar: Bloomsbury USA (Similarity: 89%)
  - Similar: Bloomsbury (Similarity: 87%)
  - Similar: Bloomsbury (NYC) (Similarity: 81%)
Publisher: Arcadia Publishing
  - Similar: Arcadia Publishing (SC) (Similarity: 92%)
  - Similar: Arcade Publishing (Similarity: 91%)
  - Similar: AVA Publishing (Similarity: 81%)
Publisher: American University in Cairo Press
  - Similar: University Press of America (Similarity: 85%)
  - Similar: Dante University of America Press (Similarity: 81%)
Publisher: HarperCollins Publishers Ltd
  - Similar: HarperCollins Publishers (Similarity: 92%)
  - Similar: HarperCollinsPublishers (Similarity: 90%)
  - Similar: HarperCollins Publishers  Inc. (Similarity: 89%)
Publisher: Scholastic  Inc.
  - Similar: Scholastic Inc. (Similarity: 100%)
  - Similar: Scholastic Inc (Similarity: 100%)
  - Similar: Scholastic (Similarity: 83%)
  - Similar: Hippo/Scholastic (Similarity: 80%)
Publisher: Penguin Books (NY)
  - Similar: Penguin Books (NYC) (Similarity: 97%)
  - Similar: Penguin Books (Similarity: 90%)
  - Similar: Penguin Books Ltd (Similarity: 85%)
  - Similar: Penguin Books Ltd. (Similarity: 85%)
  - Similar: Penguin Books Canada (Similarity: 83%)
  - Similar: Penguin Books (London) (Similarity: 83%)
  - Similar: Penguin Books USA Inc. (Similarity: 81%)
Publisher: Random House (NY)
  - Similar: Random House  Inc. (Similarity: 90%)
  - Similar: Random House Inc. (Similarity: 90%)
  - Similar: Random House (Similarity: 89%)
  - Similar: Random house (Similarity: 89%)
Publisher: Atlantic Books
  - Similar: Atlantic Books (UK) (Similarity: 90%)
  - Similar: North Atlantic Books (Similarity: 82%)
  - Similar: Ballantine Books (Similarity: 80%)
  - Similar: BALLANTINE BOOKS (Similarity: 80%)
Publisher: Penguin Group
  - Similar: Penguin Group(CA) (Similarity: 90%)
  - Similar: Penguin Group (USA) (Similarity: 87%)
  - Similar: Puffin Group (Similarity: 80%)
Publisher: Scribner Classics
  - Similar: Signet Classics (Similarity: 81%)
Publisher: University of South Carolina Press
  - Similar: University of North Carolina Press (Similarity: 82%)
  - Similar: University of Arizona Press (Similarity: 82%)
  - Similar: University of California Press (Similarity: 81%)
Publisher: Audio Partners
  - Similar: The Audio Partners (Similarity: 88%)
Publisher: St. Martins Press-3PL
  - Similar: St. Martins Press (Similarity: 89%)
  - Similar: St. Martin's Press (Similarity: 81%)
Publisher: HarperCollins UK
  - Similar: HarperCollins (Similarity: 90%)
  - Similar: Harpercollins (Similarity: 90%)
Publisher: Pelican Publishing
  - Similar: Pan Publishing (Similarity: 88%)
  - Similar: Merlin Publishing (Similarity: 86%)
  - Similar: Kaplan Publishing (Similarity: 86%)
  - Similar: Planeta Publishing (Similarity: 83%)
  - Similar: Delta Publishing (Similarity: 82%)
  - Similar: Pelican Publishing Company (Similarity: 82%)
  - Similar: Alliance Publishing (Similarity: 81%)
  - Similar: Pippin Publishing (Similarity: 80%)
Publisher: Douglas  McIntyre
  - Similar: Douglas & McIntyre (Similarity: 100%)
Publisher: Running Press
  - Similar: Running Press Kids (Similarity: 84%)
  - Similar: Running Press Adult (Similarity: 81%)
Publisher: Time-Life Books  Inc.
  - Similar: Time Life Books (Similarity: 88%)
  - Similar: Time-Life Books (Similarity: 88%)
Publisher: Walker Books
  - Similar: Walker Books Ltd (Similarity: 86%)
  - Similar: Warner Books (Similarity: 83%)
  - Similar: Gallery Books (Similarity: 80%)
Publisher: Harper Perennial (NYC)
  - Similar: Harper Perennial (Similarity: 89%)
  - Similar: HarperPerennial (Similarity: 86%)
Publisher: Viking Children's Books
  - Similar: Sterling Children's Books (Similarity: 83%)
Publisher: Collectors Library
  - Similar: Collector's Library (Similarity: 92%)
Publisher: Philomel Books
  - Similar: Plume Books (Similarity: 80%)
Publisher: Paulist Press
  - Similar: Paris Press (Similarity: 83%)
  - Similar: Polity Press (Similarity: 80%)
Publisher: Chelsea House Publications
  - Similar: Chelsea House Publishers (Similarity: 80%)
Publisher: Wizards of the Coast
  - Similar: TSR Inc./Wizards of the Coast (Similarity: 83%)
Publisher: MacMillan Audio
  - Similar: Macmillan Audio (Similarity: 100%)
Publisher: Barron's Educational Series
  - Similar: Barrons Educational Series (Similarity: 94%)
Publisher: HarperPerennial / Perennial Classics
  - Similar: Harper Perennial Modern Classics (Similarity: 85%)
  - Similar: Harper Perennial Classics (Similarity: 85%)
Publisher: Harper Perennial Classics
  - Similar: Harper Perennial Modern Classics (Similarity: 88%)
  - Similar: HarperPerennial / Perennial Classics (Similarity: 85%)
  - Similar: Ecco/Harper Perennial (Similarity: 83%)
Publisher: The Crossroad Publishing Company
  - Similar: Crossroad Publishing Company (Similarity: 93%)
Publisher: Ivan R. Dee Publisher
  - Similar: Ivan R. Dee  Publisher (Similarity: 100%)
Publisher: Delta Publishing
  - Similar: Dell Publishing (Similarity: 90%)
  - Similar: DK Publishing (Similarity: 83%)
  - Similar: Planeta Publishing (Similarity: 82%)
  - Similar: Pelican Publishing (Similarity: 82%)
  - Similar: Deodand Publishing (Similarity: 82%)
  - Similar: AVA Publishing (Similarity: 80%)
  - Similar: Pan Publishing (Similarity: 80%)
  - Similar: IDW Publishing (Similarity: 80%)
Publisher: Plume (Penguin Books Ltd)
  - Similar: Penguin Books Ltd (Similarity: 85%)
  - Similar: Penguin Books Ltd. (Similarity: 85%)
  - Similar: Penguin Books  Limited (UK) (Similarity: 81%)
Publisher: Random House: Modern Library
  - Similar: Modern Library/Random House (NY) (Similarity: 95%)
Publisher: Gemstone Publishing
  - Similar: Prestel Publishing (Similarity: 81%)
  - Similar: Blackstone Publishing (Similarity: 80%)
Publisher: Golden Books
  - Similar: Gollehon Books (Similarity: 85%)
Publisher: Running Press Kids
  - Similar: Running Press (Similarity: 84%)
  - Similar: Running Press Adult (Similarity: 81%)
Publisher: Puffin Books
  - Similar: Puffin Audiobooks (Similarity: 83%)
  - Similar: Penguin Books (Similarity: 80%)
Publisher: Baen Books
  - Similar: Baker Books (Similarity: 86%)
  - Similar: Ace Books (Similarity: 84%)
  - Similar: Bantam Books (Similarity: 82%)
  - Similar: B Books (Similarity: 82%)
  - Similar: Avenel Books (Similarity: 82%)
  - Similar: Avon Books (Similarity: 80%)
Publisher: William Morrow & Company  Inc.
  - Similar: William Morrow & Company (Similarity: 92%)
  - Similar: William Morrow & Company (NYC) (Similarity: 85%)
Publisher: Hodder & Stoughton Ltd
  - Similar: Hodder & Stoughton (Similarity: 89%)
  - Similar: Hodder and Stoughton (Similarity: 80%)
Publisher: University of Georgia Press
  - Similar: University Press of Florida (Similarity: 89%)
  - Similar: University Press of America (Similarity: 89%)
  - Similar: University of California Press (Similarity: 84%)
  - Similar: University of Nebraska Press (Similarity: 84%)
  - Similar: University of Minnesota Press (Similarity: 82%)
  - Similar: University Of Minnesota Press (Similarity: 82%)
  - Similar: University of Chicago Press (Similarity: 81%)
  - Similar: University Of Chicago Press (Similarity: 81%)
  - Similar: University of Calgary Press (Similarity: 81%)
  - Similar: University of Notre Dame Press (Similarity: 81%)
  - Similar: University of Arizona Press (Similarity: 81%)
  - Similar: University of Illinois Press (Similarity: 80%)
  - Similar: University of Michigan Press (Similarity: 80%)
  - Similar: University of Arkansas Press (Similarity: 80%)
  - Similar: Dante University of America Press (Similarity: 80%)
Publisher: French & European Publications
  - Similar: French & European Pubns (Similarity: 86%)
Publisher: Orbis Books
  - Similar: Orbit Books (Similarity: 91%)
  - Similar: Orb Books (Similarity: 90%)
  - Similar: Corgi Books (Similarity: 82%)
  - Similar: Orion Books (Similarity: 82%)
  - Similar: Tor Books (Similarity: 80%)
Publisher: University of Washington Press
  - Similar: University of Utah Press (Similarity: 81%)
  - Similar: University of Wisconsin Press (Similarity: 81%)
Publisher: Bradford Book
  - Similar: Burford Books (Similarity: 85%)
Publisher: W.W. Norton & Company (NYC)
  - Similar: W.W. Norton & Company (NY) (Similarity: 98%)
  - Similar: W. W. Norton & Company (Similarity: 90%)
  - Similar: W. W. Norton  Company (Similarity: 90%)
  - Similar: W.W. Norton & Company (Similarity: 90%)
  - Similar: W.W. Norton  Company (Similarity: 90%)
  - Similar: W.W. Norton & Company  Inc. (NY) (Similarity: 89%)
  - Similar: W.W. Norton & Company (NY/London) (Similarity: 84%)
  - Similar: W. W. Norton and Company (Similarity: 82%)
Publisher: Penguin Books  Limited (UK)
  - Similar: Penguin Books Limited (Similarity: 93%)
  - Similar: Penguin Book Limited (Similarity: 91%)
  - Similar: Penguin Books Ltd (Similarity: 83%)
  - Similar: Penguin Books Ltd. (Similarity: 83%)
  - Similar: Plume (Penguin Books Ltd) (Similarity: 81%)
Publisher: Titan Books
  - Similar: Titan Books Ltd (Similarity: 85%)
  - Similar: It Books (Similarity: 84%)
  - Similar: Pan Books (Similarity: 80%)
Publisher: Apress
  - Similar: AK Press (Similarity: 86%)
  - Similar: Fab Press (Similarity: 80%)
Publisher: DAW Hardcover
  - Similar: Ace Hardcover (Similarity: 85%)
Publisher: Miramax Books
  - Similar: Mira Books (Similarity: 87%)
  - Similar: Mira books (Similarity: 87%)
Publisher: St. Martin's Paperbacks
  - Similar: St. Martin's Paperback (Similarity: 98%)
Publisher: Little  Brown Books for Young Readers
  - Similar: Little  Brown Young Readers (Similarity: 84%)
Publisher: The New York Review Children's Collection
  - Similar: New York Review Children's Collection (Similarity: 95%)
Publisher: Random House Children's Books
  - Similar: Hodder Children's Books (Similarity: 81%)
Publisher: Simon Publications
  - Similar: Sentient Publications (Similarity: 82%)
  - Similar: Wisdom Publications (Similarity: 81%)
Publisher: Chatto & Windus
  - Similar: Chatto  Windus (Similarity: 100%)
  - Similar: Chatto and Windus (Similarity: 87%)
Publisher: Le Livre de Poche
  - Similar: Livre de Poche (Similarity: 90%)
Publisher: Harvest House Publishers
  - Similar: Arkham House Publishers (Similarity: 81%)
Publisher: TarcherPerigee
  - Similar: Tarcherperigee (Similarity: 100%)
Publisher: Penguin Group(CA)
  - Similar: Penguin Group (Similarity: 90%)
Publisher: Harper Trophy
  - Similar: HarperTrophy (Similarity: 96%)
Publisher: Vintage Books
  - Similar: Vintage Books USA (Similarity: 87%)
  - Similar: Viking Books (Similarity: 80%)
  - Similar: Living Books (Similarity: 80%)
Publisher: Doubleday Publishing (NY)
  - Similar: Knopf Doubleday Publishing Group (Similarity: 80%)
Publisher: Hay House
  - Similar: Holiday House (Similarity: 82%)
Publisher: Penguin Books /Penguin Classics
  - Similar: Penguin Books (Penguin Classics) (Similarity: 100%)
  - Similar: Penguin Books/Penguin Popular Classics (Similarity: 88%)
Publisher: Faber & Faber
  - Similar: Faber  Faber (Similarity: 100%)
  - Similar: Faber and Faber (Similarity: 85%)
  - Similar: Faber & Faber Ltd. (Similarity: 85%)
Publisher: G. P. Putnam's Sons
  - Similar: G.P. Putnam's Sons (Similarity: 100%)
Publisher: HarperSanFrancisco
  - Similar: HarperCollins (SanFrancisco) (Similarity: 82%)
Publisher: Knopf Books for Young Readers
  - Similar: Alfred A. Knopf Books for Young Readers (Similarity: 87%)
  - Similar: HMH Books for Young Readers (Similarity: 86%)
  - Similar: Random House Books for Young Readers (Similarity: 80%)
Publisher: Penguin (Non-Classics)
  - Similar: Penguin Classics (Similarity: 89%)
  - Similar: Penguin Classic (Similarity: 86%)
Publisher: Anchor Books/Knopf Doubleday Publishing Group
  - Similar: Knopf Doubleday Publishing Group (Similarity: 83%)
Publisher: Debolsillo
  - Similar: DeBolsillo (Similarity: 100%)
  - Similar: DEBOLS!LLO (Similarity: 90%)
Publisher: French & European Pubns
  - Similar: French & European Publications (Similarity: 86%)
  - Similar: French & European (Similarity: 83%)
Publisher: Limitless Corporation
  - Similar: Xlibris Corporation (Similarity: 80%)
Publisher: University of Michigan Press
  - Similar: University of Chicago Press (Similarity: 87%)
  - Similar: University Of Chicago Press (Similarity: 87%)
  - Similar: University Press of America (Similarity: 87%)
  - Similar: University of Calgary Press (Similarity: 84%)
  - Similar: University of New Mexico Press (Similarity: 83%)
  - Similar: University of Illinois Press (Similarity: 82%)
  - Similar: Dante University of America Press (Similarity: 82%)
  - Similar: University Press of Kansas (Similarity: 81%)
  - Similar: Indiana University Press (Similarity: 81%)
  - Similar: University of Minnesota Press (Similarity: 81%)
  - Similar: University Of Minnesota Press (Similarity: 81%)
  - Similar: University of Georgia Press (Similarity: 80%)
  - Similar: University Press of Florida (Similarity: 80%)
  - Similar: University of Arizona Press (Similarity: 80%)
Publisher: Fourth Estate
  - Similar: Fourth Estate (GB) (Similarity: 90%)
  - Similar: Fourth Estate Ltd (Similarity: 87%)
Publisher: AudioGO
  - Similar: Audiogo (Similarity: 100%)
Publisher: Random House Anchor
  - Similar: Random House Audio (Similarity: 81%)
Publisher: Pantheon Books
  - Similar: Pantheon Books (NY) (Similarity: 90%)
  - Similar: Lantern Books (Similarity: 81%)
Publisher: Phoenix Press (UK)
  - Similar: Phoenix Press (Similarity: 90%)
Publisher: Sage Publications  Inc
  - Similar: Sage Publications Ltd (Similarity: 86%)
  - Similar: F & W Publications Inc. (Similarity: 83%)
Publisher: Faber and Faber
  - Similar: Faber  Faber (Similarity: 85%)
  - Similar: Faber & Faber (Similarity: 85%)
Publisher: Island Press
  - Similar: Amistad Press (Similarity: 80%)
Publisher: Del Rey/Ballantine Books
  - Similar: Ballantine Books (NY) (Similarity: 84%)
  - Similar: Ballantine Books (Similarity: 80%)
  - Similar: BALLANTINE BOOKS (Similarity: 80%)
Publisher: Ballantine Books (NY)
  - Similar: Ballantine Books (Similarity: 91%)
  - Similar: BALLANTINE BOOKS (Similarity: 91%)
  - Similar: Del Rey/Ballantine Books (Similarity: 84%)
Publisher: Lantern Books
  - Similar: Pantheon Books (Similarity: 81%)
  - Similar: Warner Books (Similarity: 80%)
Publisher: BALLANTINE BOOKS
  - Similar: Ballantine Books (Similarity: 100%)
  - Similar: Ballantine Books (NY) (Similarity: 91%)
  - Similar: Atlantic Books (Similarity: 80%)
  - Similar: Del Rey/Ballantine Books (Similarity: 80%)
Publisher: Lucent Books
  - Similar: Luna Books (Similarity: 82%)
Publisher: Villard Books
  - Similar: Island Books (Similarity: 80%)
Publisher: Conari Press
  - Similar: Oni Press (Similarity: 86%)
  - Similar: Clarity Press (Similarity: 80%)
  - Similar: Council Press (Similarity: 80%)
Publisher: Penguin Classic
  - Similar: Penguin Classics (Similarity: 97%)
  - Similar: Penguin (Non-Classics) (Similarity: 86%)
Publisher: Harpperen
  - Similar: HarperTeen (Similarity: 84%)
  - Similar: Harper (Similarity: 80%)
  - Similar: Harper Teen (Similarity: 80%)
Publisher: Modern Library/Random House (NY)
  - Similar: Random House: Modern Library (Similarity: 95%)
Publisher: Columbia University Press
  - Similar: Cambridge University Press (Similarity: 82%)
  - Similar: Cornell University Press (Similarity: 82%)
Publisher: Lectorum Publications
  - Similar: Lerner Publications (Similarity: 80%)
Publisher: NYRB Classics
  - Similar: Tor Classics (Similarity: 80%)
Publisher: Penguin Books/Penguin Popular Classics
  - Similar: Penguin Books (Penguin Classics) (Similarity: 88%)
  - Similar: Penguin Books /Penguin Classics (Similarity: 88%)
Publisher: Northwestern University Press
  - Similar: Northeastern University Press (Similarity: 97%)
  - Similar: Northwestern Univ Press (Similarity: 88%)
  - Similar: Manchester University Press (Similarity: 86%)
Publisher: HarperTrophy
  - Similar: Harper Trophy (Similarity: 96%)
Publisher: Orb Books
  - Similar: Orbis Books (Similarity: 90%)
  - Similar: Orbit Books (Similarity: 90%)
  - Similar: Tor Books (Similarity: 89%)
  - Similar: Corgi Books (Similarity: 80%)
  - Similar: Forge Books (Similarity: 80%)
  - Similar: Honor Books (Similarity: 80%)
  - Similar: Urban Books (Similarity: 80%)
  - Similar: Orion Books (Similarity: 80%)
Publisher: Forge Books
  - Similar: Courage Books (Similarity: 83%)
  - Similar: Corgi Books (Similarity: 82%)
  - Similar: Tor Books (Similarity: 80%)
  - Similar: Orb Books (Similarity: 80%)
Publisher: F & W Publications Inc.
  - Similar: Sage Publications  Inc (Similarity: 83%)
  - Similar: Dover Publications  Inc. (Similarity: 81%)
Publisher: Macmillan Audio
  - Similar: MacMillan Audio (Similarity: 100%)
Publisher: Egmont Children's Books
  - Similar: Orion Children's Books (Similarity: 84%)
  - Similar: Dutton Children's Books (Similarity: 83%)
Publisher: Random House for Young Readers
  - Similar: Random House  for Young Readers (Similarity: 100%)
  - Similar: Random House Books for Young Readers (Similarity: 91%)
Publisher: Random House  for Young Readers
  - Similar: Random House for Young Readers (Similarity: 100%)
  - Similar: Random House Books for Young Readers (Similarity: 91%)
Publisher: Tommy Nelson
  - Similar: Thomas Nelson (Similarity: 80%)
Publisher: Doubleday Books for Young Readers
  - Similar: Dutton Books for Young Readers (Similarity: 83%)
  - Similar: Delacorte Books for Young Readers (Similarity: 82%)
Publisher: University of Massachusetts Press
  - Similar: University Press of Kansas (Similarity: 81%)
Publisher: Profile Books
  - Similar: Rodale Books (Similarity: 80%)
Publisher: Counterpoint LLC
  - Similar: Counterpoint (Similarity: 86%)
Publisher: J'ai Lu
  - Similar: J'ai lu (Similarity: 100%)
  - Similar: J'AI LU (Similarity: 100%)
Publisher: Random house
  - Similar: Random House (Similarity: 100%)
  - Similar: Random House (NY) (Similarity: 89%)
  - Similar: Random House  Inc. (Similarity: 86%)
  - Similar: Random House Inc. (Similarity: 86%)
  - Similar: Random House Audio (Similarity: 80%)
  - Similar: Random House Trade (Similarity: 80%)
Publisher: Virago UK
  - Similar: Virago (Similarity: 80%)
Publisher: DK Publishing
  - Similar: IDW Publishing (Similarity: 89%)
  - Similar: Dell Publishing (Similarity: 86%)
  - Similar: Ig Publishing (Similarity: 85%)
  - Similar: Delta Publishing (Similarity: 83%)
  - Similar: AVA Publishing (Similarity: 81%)
  - Similar: Pan Publishing (Similarity: 81%)
  - Similar: P & R Publishing (Similarity: 81%)
  - Similar: Arcade Publishing (Similarity: 80%)
  - Similar: Kaplan Publishing (Similarity: 80%)
  - Similar: Modern Publishing (Similarity: 80%)
Publisher: W.W. Norton  Company
  - Similar: W. W. Norton & Company (Similarity: 100%)
  - Similar: W. W. Norton  Company (Similarity: 100%)
  - Similar: W.W. Norton & Company (Similarity: 100%)
  - Similar: W.W. Norton & Company (NY) (Similarity: 92%)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 90%)
  - Similar: W. W. Norton and Company (Similarity: 90%)
  - Similar: W. W. Norton & Co. (Similarity: 84%)
  - Similar: W.W. Norton & Company  Inc. (NY) (Similarity: 84%)
Publisher: Holt Paperbacks
  - Similar: Owlet Paperbacks (Similarity: 90%)
Publisher: Titan Books Ltd
  - Similar: Titan Books (Similarity: 85%)
Publisher: Random House Value Publishing
  - Similar: Random House Reference Publishing (Similarity: 81%)
Publisher: Little  Brown Young Readers
  - Similar: Little  Brown Books for Young Readers (Similarity: 84%)
Publisher: Albury Publishing
  - Similar: Bloomsbury Publishing (Similarity: 84%)
  - Similar: Haus Publishing (Similarity: 81%)
Publisher: Little  Brown & Company
  - Similar: Little Brown and Company (Similarity: 91%)
  - Similar: Little  Brown and Company (Similarity: 91%)
Publisher: Amherst Media  Inc.
  - Similar: Amherst Media (Similarity: 87%)
Publisher: Amherst Media
  - Similar: Amherst Media  Inc. (Similarity: 87%)
Publisher: Express Publishing
  - Similar: Prestel Publishing (Similarity: 83%)
  - Similar: Osprey Publishing (Similarity: 80%)
  - Similar: Plexus Publishing (Similarity: 80%)
Publisher: Barefoot Books
  - Similar: Baronet Books (Similarity: 81%)
Publisher: University Press of Florida
  - Similar: University of Georgia Press (Similarity: 89%)
  - Similar: University of California Press (Similarity: 88%)
  - Similar: University Press of America (Similarity: 85%)
  - Similar: University of Arizona Press (Similarity: 85%)
  - Similar: Oxford University Press (Similarity: 84%)
  - Similar: University of Illinois Press (Similarity: 84%)
  - Similar: University of Alaska Press (Similarity: 83%)
  - Similar: University of Calgary Press (Similarity: 81%)
  - Similar: University of Michigan Press (Similarity: 80%)
  - Similar: University of Arkansas Press (Similarity: 80%)
  - Similar: University of Nebraska Press (Similarity: 80%)
  - Similar: Dante University of America Press (Similarity: 80%)
  - Similar: Howard University Press (Similarity: 80%)
Publisher: New Island Books
  - Similar: Island Books (Similarity: 86%)
Publisher: Dell Publishing Company
  - Similar: Owl Publishing Company (Similarity: 89%)
  - Similar: Pelican Publishing Company (Similarity: 86%)
  - Similar: MacMillan Publishing Company (Similarity: 82%)
  - Similar: Hackett Publishing Company (Similarity: 82%)
Publisher: Transworld Publishers
  - Similar: Transworld Publishers Ltd (Similarity: 91%)
Publisher: Indiana University Press
  - Similar: University of Michigan Press (Similarity: 81%)
  - Similar: University Press of Kansas (Similarity: 80%)
Publisher: Mayfield Publishing Company (NY)
  - Similar: MacMillan Publishing Company (Similarity: 83%)
Publisher: Orchard Books
  - Similar: Orchard Books (NY) (Similarity: 90%)
  - Similar: Howard Books (Similarity: 80%)
Publisher: Abingdon Press
  - Similar: Phaidon Press (Similarity: 81%)
Publisher: Phoenix Press
  - Similar: Phoenix Press (UK) (Similarity: 90%)
  - Similar: Oni Press (Similarity: 82%)
Publisher: Hyperion Books
  - Similar: Disney Hyperion Books (Similarity: 80%)
  - Similar: Orion Books (Similarity: 80%)
Publisher: iBooks
  - Similar: rbooks (Similarity: 83%)
Publisher: Everyman's Library Classics
  - Similar: Everyman's Library (Similarity: 80%)
Publisher: Harcourt  Inc.(Harvest Book)
  - Similar: A Harvest Book/Harcourt  Inc. (Similarity: 96%)
  - Similar: Harvest Books/Harcourt (Similarity: 89%)
Publisher: McClelland and Stewart
  - Similar: McClelland & Stewart (Similarity: 90%)
Publisher: Dutton Books for Young Readers
  - Similar: Delacorte Books for Young Readers (Similarity: 83%)
  - Similar: Doubleday Books for Young Readers (Similarity: 83%)
  - Similar: HMH Books for Young Readers (Similarity: 81%)
Publisher: University of North Carolina Press
  - Similar: University of South Carolina Press (Similarity: 82%)
  - Similar: University of Arizona Press (Similarity: 82%)
  - Similar: University of California Press (Similarity: 81%)
Publisher: Lumen
  - Similar: Plume (Similarity: 80%)
Publisher: Methuen Publishing
  - Similar: Merlin Publishing (Similarity: 80%)
  - Similar: Modern Publishing (Similarity: 80%)
Publisher: Bloomsbury USA
  - Similar: Bloomsbury UK (Similarity: 89%)
  - Similar: Bloomsbury (Similarity: 83%)
Publisher: BBC Audiobooks
  - Similar: BBC Audiobooks Ltd (Similarity: 88%)
Publisher: Daw Books
  - Similar: DAW Books (Similarity: 100%)
  - Similar: DAW Books (NY) (Similarity: 86%)
  - Similar: Shaw Books (Similarity: 84%)
  - Similar: Dial Books (Similarity: 84%)
  - Similar: DAW Books Inc (Similarity: 82%)
  - Similar: Cedar Books (Similarity: 80%)
Publisher: Rowohlt Tb
  - Similar: Rowohlt Tb. (Similarity: 100%)
  - Similar: Rowohlt (Similarity: 82%)
Publisher: Arbordale Publishing
  - Similar: Arcade Publishing (Similarity: 81%)
Publisher: Virgin Publishing
  - Similar: Regnery Publishing (Similarity: 80%)
Publisher: Taunton Press
  - Similar: Sunstone Press (Similarity: 81%)
Publisher: Houghton Mifflin Harcourt P
  - Similar: Houghton Mifflin Harcourt (Similarity: 96%)
  - Similar: Rux Martin/Houghton Mifflin Harcourt (Similarity: 83%)
Publisher: DeBolsillo
  - Similar: Debolsillo (Similarity: 100%)
  - Similar: DEBOLS!LLO (Similarity: 90%)
Publisher: Orion Publishing Co.
  - Similar: Orion Publishing Co (Similarity: 100%)
  - Similar: Orion Publishing (Similarity: 91%)
  - Similar: Orion Publishing Group (Similarity: 88%)
  - Similar: Howard Publishing Co (Similarity: 82%)
  - Similar: Orion Publishing Group  Ltd. (Similarity: 80%)
Publisher: Schirmer/Mosel
  - Similar: Schirmer Mosel (Similarity: 100%)
Publisher: Michael O'Mara
  - Similar: Michael O'Mara Books (Similarity: 82%)
Publisher: Baronet Books
  - Similar: Barefoot Books (Similarity: 81%)
  - Similar: Bantam Books (Similarity: 80%)
Publisher: Plaza y Janés
  - Similar: Plaza y Janes (Similarity: 96%)
  - Similar: Plaza & Janés (Similarity: 91%)
Publisher: Inter Varsity Press
  - Similar: Inter-Varsity Press (Similarity: 100%)
Publisher: Random House  Inc.
  - Similar: Random House Inc. (Similarity: 100%)
  - Similar: Random House (NY) (Similarity: 90%)
  - Similar: Random House (Similarity: 86%)
  - Similar: Random house (Similarity: 86%)
Publisher: MacMillan Publishing Company
  - Similar: Mayfield Publishing Company (NY) (Similarity: 83%)
  - Similar: Dell Publishing Company (Similarity: 82%)
  - Similar: Pelican Publishing Company (Similarity: 81%)
  - Similar: Owl Publishing Company (Similarity: 80%)
  - Similar: Macmillan Publishing Company/Collier Books (Similarity: 80%)
Publisher: Grove Press (NYC)
  - Similar: Grove Press (Similarity: 85%)
Publisher: Arrow Books
  - Similar: Arrow Books Ltd (Similarity: 85%)
Publisher: Games Workshop(uk)
  - Similar: Games Workshop (Similarity: 90%)
Publisher: Fleming H Revell Co
  - Similar: Fleming H. Revell Company (Similarity: 88%)
Publisher: Living Books
  - Similar: Viking Books (Similarity: 83%)
  - Similar: Vintage Books (Similarity: 80%)
  - Similar: Inkling Books (Similarity: 80%)
Publisher: Hill & Wang
  - Similar: Hill and Wang (Similarity: 82%)
Publisher: Schocken Books Inc
  - Similar: Schocken Books (Similarity: 88%)
Publisher: Hill and Wang
  - Similar: Hill & Wang (Similarity: 82%)
Publisher: Gürze Books
  - Similar: Gurze Books (Similarity: 95%)
Publisher: Gurze Books
  - Similar: Gürze Books (Similarity: 95%)
Publisher: Editions du Rocher
  - Similar: Éditions du Rocher (Similarity: 80%)
Publisher: Harpercollins Children's Books
  - Similar: Harpercollins Childrens Books (Similarity: 95%)
  - Similar: Harcourt Children's Books (Similarity: 80%)
Publisher: New Directions Publishing Corporation
  - Similar: New Directions Publishing (Similarity: 81%)
Publisher: Western Publishing Company  Golden Books
  - Similar: Book Publishing Company (TN) (Similarity: 80%)
Publisher: University of Texas Press
  - Similar: Texas A&M University Press (Similarity: 90%)
  - Similar: Rutgers University Press (Similarity: 82%)
  - Similar: University of Toronto Press (Similarity: 81%)
  - Similar: Ohio State University Press (Similarity: 81%)
Publisher: Editions 10/18
  - Similar: Éditions 10/18 (Similarity: 96%)
Publisher: Bookspan
  - Similar: Pan Books (Similarity: 94%)
Publisher: Th1nk Books
  - Similar: HQN Books (Similarity: 80%)
Publisher: Éditions du Rocher
  - Similar: Editions du Rocher (Similarity: 80%)
Publisher: Blanvalet Taschenbuch
  - Similar: Blanvalet Taschenbuch Verlag (Similarity: 86%)
Publisher: NAL Books
  - Similar: Seal Books (Similarity: 84%)
  - Similar: Luna Books (Similarity: 84%)
  - Similar: Dial Books (Similarity: 84%)
  - Similar: Regal Books (Similarity: 80%)
Publisher: Plaza & Janes Editores Sa
  - Similar: Plaza & Janes Editores  S.A. (Similarity: 94%)
  - Similar: Plaza & Janés Editores  S.A. (Similarity: 91%)
Publisher: Sutton
  - Similar: Dutton (Similarity: 83%)
Publisher: George Braziller
  - Similar: George Braziller Inc. (Similarity: 89%)
Publisher: Amistad Press
  - Similar: Naiad Press (Similarity: 83%)
  - Similar: MIT Press (Similarity: 82%)
  - Similar: Island Press (Similarity: 80%)
  - Similar: MIT Press (MA) (Similarity: 80%)
Publisher: TusQuets
  - Similar: Tusquets (Similarity: 100%)
Publisher: Teacher's Pet Publications  Inc.
  - Similar: Teacher's Pet Publications (Similarity: 93%)
Publisher: Gallimard Education
  - Similar: Editions Gallimard (Similarity: 86%)
Publisher: Hamish Hamilton
  - Similar: Hamish hamilton (Similarity: 100%)
  - Similar: Hamish Hamilton Ltd (Similarity: 88%)
Publisher: Bantam Spectra (NY)
  - Similar: Bantam Spectra (Similarity: 90%)
  - Similar: BantamSpectra (Similarity: 87%)
  - Similar: Spectra/Bantam Books (NYC) (Similarity: 83%)
Publisher: Pocket Books/Simon & Schuster (NY)
  - Similar: Pocket Books / Simon & Schuster  Inc. (Similarity: 95%)
Publisher: Little Brown
  - Similar: Little  Brown (Similarity: 100%)
Publisher: William Morrow / Harper
  - Similar: William Morrow (Similarity: 80%)
Publisher: Norma Editorial
  - Similar: Editorial RM (Similarity: 89%)
  - Similar: Roca Editorial (Similarity: 83%)
Publisher: Oberon Books
  - Similar: Orion Books (Similarity: 87%)
Publisher: Owlet Paperbacks
  - Similar: Holt Paperbacks (Similarity: 90%)
Publisher: HarperPerennial
  - Similar: Harper Perennial (Similarity: 97%)
  - Similar: Harper Perennial (NYC) (Similarity: 86%)
  - Similar: Ecco/Harper Perennial (Similarity: 83%)
Publisher: Dutton Children's Books
  - Similar: Orion Children's Books (Similarity: 84%)
  - Similar: Egmont Children's Books (Similarity: 83%)
Publisher: Owl Books
  - Similar: Owl Books (NY) (Similarity: 86%)
Publisher: Owl Books (NY)
  - Similar: Owl Books (Similarity: 86%)
Publisher: Aris and Phillips
  - Similar: Aris & Phillips (Similarity: 87%)
Publisher: Dover Publications  Inc.
  - Similar: Dover Publications (NYC) (Similarity: 95%)
  - Similar: Dover Publications (NY) (Similarity: 93%)
  - Similar: Dover Publications (Similarity: 90%)
  - Similar: F & W Publications Inc. (Similarity: 81%)
Publisher: Stanford University Press (Stanford  CA)
  - Similar: Stanford University Press (Similarity: 81%)
Publisher: Bloomsbury (NYC)
  - Similar: Bloomsbury (Similarity: 83%)
  - Similar: Bloomsbury UK (Similarity: 81%)
Publisher: Random House Vintage Books
  - Similar: Random House Vintage (Similarity: 87%)
  - Similar: Random House (Vintage) (Similarity: 87%)
  - Similar: Vintage/Random House (NY) (Similarity: 82%)
Publisher: Dafina Books
  - Similar: Gaia Books (Similarity: 82%)
  - Similar: DAW Books Inc (Similarity: 80%)
Publisher: Dell Publishing
  - Similar: Delta Publishing (Similarity: 90%)
  - Similar: DK Publishing (Similarity: 86%)
  - Similar: IDW Publishing (Similarity: 83%)
  - Similar: Arcade Publishing (Similarity: 81%)
  - Similar: Merlin Publishing (Similarity: 81%)
  - Similar: Modern Publishing (Similarity: 81%)
Publisher: Nation Books
  - Similar: Creation Books (Similarity: 85%)
  - Similar: Reaktion Books (Similarity: 85%)
  - Similar: Icon Books (Similarity: 82%)
  - Similar: Clarion Books (Similarity: 80%)
Publisher: AK Press
  - Similar: Apress (Similarity: 86%)
  - Similar: Fab Press (Similarity: 82%)
  - Similar: Annick Press (Similarity: 80%)
Publisher: Serpent's Tail
  - Similar: Serpents Tail (Similarity: 89%)
Publisher: Odonian Press
  - Similar: Oni Press (Similarity: 82%)
  - Similar: Dominion Press (Similarity: 81%)
Publisher: Fawcett Crest Books
  - Similar: Fawcett Books (Similarity: 81%)
  - Similar: Fawcett crest (Similarity: 81%)
  - Similar: Fawcett Crest (Similarity: 81%)
Publisher: Virago
  - Similar: Virago UK (Similarity: 80%)
Publisher: Barnes & Noble Books
  - Similar: Barn Owl Books (Similarity: 81%)
  - Similar: Barnes & Noble (Similarity: 80%)
  - Similar: Barnes  Noble (Similarity: 80%)
Publisher: Fourth Estate (GB)
  - Similar: Fourth Estate (Similarity: 90%)
  - Similar: Fourth Estate Ltd (Similarity: 85%)
Publisher: W. W. Norton & Co.
  - Similar: W.W. Norton (Similarity: 87%)
  - Similar: W. W. Norton & Company (Similarity: 84%)
  - Similar: W. W. Norton  Company (Similarity: 84%)
  - Similar: W.W. Norton & Company (Similarity: 84%)
  - Similar: W.W. Norton  Company (Similarity: 84%)
Publisher: Bantam Classic
  - Similar: Bantam Classics (Similarity: 97%)
Publisher: Signet Classic
  - Similar: Signet Classics (Similarity: 97%)
Publisher: Cooper Square Publishers
  - Similar: Square One Publishers (Similarity: 89%)
Publisher: TIme Warner Books
  - Similar: Time Warner Books UK (Similarity: 92%)
  - Similar: Warner Books (Similarity: 83%)
  - Similar: Warner Books (NY) (Similarity: 81%)
  - Similar: Mariner Books (Similarity: 80%)
Publisher: Laurel-Leaf Books
  - Similar: Laurel Leaf Books (Similarity: 100%)
Publisher: Moody Publishers (Chicago)
  - Similar: Moody Publishers (Similarity: 80%)
Publisher: Phaidon Press
  - Similar: Abingdon Press (Similarity: 81%)
  - Similar: Poseidon Press (Similarity: 81%)
Publisher: Thames & Hudson Ltd
  - Similar: Thames  Hudson (Similarity: 87%)
  - Similar: Thames & Hudson (Similarity: 87%)
Publisher: Eos
  - Similar: EOS (Similarity: 100%)
Publisher: Hqn
  - Similar: HQN (Similarity: 100%)
Publisher: Penguin Books Canada
  - Similar: Penguin Books (NY) (Similarity: 83%)
  - Similar: Penguin Books Ltd (Similarity: 81%)
  - Similar: Penguin Books (NYC) (Similarity: 81%)
  - Similar: Penguin Books Ltd. (Similarity: 81%)
  - Similar: Penguin Books (London) (Similarity: 80%)
Publisher: Harvard University Press (Cambridge)
  - Similar: Cambridge University Press (Similarity: 87%)
  - Similar: Harvard University Press (Similarity: 83%)
Publisher: Fireside Books
  - Similar: Firebird Books (Similarity: 86%)
Publisher: VIZ Media
  - Similar: Viz Media (Similarity: 100%)
  - Similar: VIZ Media LLC (Similarity: 82%)
Publisher: J'ai lu
  - Similar: J'ai Lu (Similarity: 100%)
  - Similar: J'AI LU (Similarity: 100%)
Publisher: Poseidon Press
  - Similar: Phaidon Press (Similarity: 81%)
Publisher: Meisha Merlin Publishing
  - Similar: Merlin Publishing (Similarity: 83%)
Publisher: St. Martins Press
  - Similar: St. Martin's Press (Similarity: 91%)
  - Similar: St. Martins Press-3PL (Similarity: 89%)
  - Similar: St. Martin's (Similarity: 81%)
  - Similar: St. Augustines Press (Similarity: 80%)
Publisher: New Press
  - Similar: ECW Press (Similarity: 89%)
  - Similar: New Press (NY) (Similarity: 86%)
  - Similar: The New Press (Similarity: 82%)
  - Similar: New Press  The (Similarity: 82%)
Publisher: TokyoPop
  - Similar: Tokyopop (Similarity: 100%)
Publisher: Viz Media
  - Similar: VIZ Media (Similarity: 100%)
  - Similar: VIZ Media LLC (Similarity: 82%)
Publisher: EOS
  - Similar: Eos (Similarity: 100%)
Publisher: Random House Vintage
  - Similar: Random House (Vintage) (Similarity: 100%)
  - Similar: Vintage/Random House (NY) (Similarity: 93%)
  - Similar: Random House Vintage Books (Similarity: 87%)
  - Similar: Random House Trade (Similarity: 84%)
  - Similar: Vintage/Random House Inc. (NYC) (Similarity: 83%)
Publisher: BantamSpectra
  - Similar: Bantam Spectra (Similarity: 96%)
  - Similar: Bantam Spectra (NY) (Similarity: 87%)
Publisher: University of Toronto Press
  - Similar: University of Texas Press (Similarity: 81%)
Publisher: Games Workshop
  - Similar: Games Workshop(uk) (Similarity: 90%)
Publisher: DAW Books
  - Similar: Daw Books (Similarity: 100%)
  - Similar: DAW Books (NY) (Similarity: 86%)
  - Similar: Shaw Books (Similarity: 84%)
  - Similar: Dial Books (Similarity: 84%)
  - Similar: DAW Books Inc (Similarity: 82%)
  - Similar: Cedar Books (Similarity: 80%)
Publisher: Luna Books
  - Similar: Lucas Books (Similarity: 86%)
  - Similar: NAL Books (Similarity: 84%)
  - Similar: Lucent Books (Similarity: 82%)
  - Similar: Lark Books (Similarity: 80%)
Publisher: Digital Manga Publishing
  - Similar: Digital Manga Publishing / Dark Horse (Similarity: 81%)
Publisher: University of Minnesota Press
  - Similar: University Of Minnesota Press (Similarity: 100%)
  - Similar: Univ Of Minnesota Press (Similarity: 88%)
  - Similar: University Press of Kansas (Similarity: 84%)
  - Similar: University of Nebraska Press (Similarity: 84%)
  - Similar: University Press of America (Similarity: 82%)
  - Similar: University of Arizona Press (Similarity: 82%)
  - Similar: University of Illinois Press (Similarity: 81%)
  - Similar: University of Michigan Press (Similarity: 81%)
  - Similar: University of Arkansas Press (Similarity: 81%)
  - Similar: University of Notre Dame Press (Similarity: 81%)
  - Similar: University of New Mexico Press (Similarity: 81%)
  - Similar: University of Alaska Press (Similarity: 80%)
Publisher: Library of American Comics
  - Similar: Library of America (Similarity: 82%)
Publisher: Orion Paperbacks
  - Similar: Hyperion Paperbacks (Similarity: 86%)
Publisher: Amereon Ltd
  - Similar: Amereon Limited (Similarity: 85%)
Publisher: Harvill Press
  - Similar: The Harvill Press (Similarity: 87%)
Publisher: Wenner Books
  - Similar: Warner Books (Similarity: 83%)
  - Similar: Weiser Books (Similarity: 83%)
  - Similar: Encounter Books (Similarity: 81%)
Publisher: Hodder & Stoughton
  - Similar: Hodder & Stoughton Ltd (Similarity: 89%)
  - Similar: Hodder and Stoughton (Similarity: 89%)
Publisher: Bantam Doubleday Dell Publishing Group
  - Similar: Bantam Doubleday Dell Publishing Company Inc. (Similarity: 80%)
Publisher: Kent State University Press (OH)
  - Similar: Kent State University Press (Similarity: 95%)
  - Similar: Ohio State University Press (Similarity: 88%)
  - Similar: Oregon State University Press (Similarity: 85%)
  - Similar: State University of New York Press (Similarity: 81%)
Publisher: Focal Press
  - Similar: Fab Press (Similarity: 80%)
Publisher: J'AI LU
  - Similar: J'ai Lu (Similarity: 100%)
  - Similar: J'ai lu (Similarity: 100%)
Publisher: AVA Publishing
  - Similar: Pan Publishing (Similarity: 86%)
  - Similar: Arcade Publishing (Similarity: 84%)
  - Similar: Kaplan Publishing (Similarity: 84%)
  - Similar: Haus Publishing (Similarity: 83%)
  - Similar: Planeta Publishing (Similarity: 81%)
  - Similar: Arcadia Publishing (Similarity: 81%)
  - Similar: DK Publishing (Similarity: 81%)
  - Similar: Ig Publishing (Similarity: 81%)
  - Similar: Delta Publishing (Similarity: 80%)
Publisher: Inkling Books
  - Similar: Viking Books (Similarity: 88%)
  - Similar: Living Books (Similarity: 80%)
Publisher: Pan Publishing
  - Similar: Kaplan Publishing (Similarity: 90%)
  - Similar: Planeta Publishing (Similarity: 88%)
  - Similar: Pelican Publishing (Similarity: 88%)
  - Similar: AVA Publishing (Similarity: 86%)
  - Similar: P & R Publishing (Similarity: 86%)
  - Similar: Pippin Publishing (Similarity: 84%)
  - Similar: Haus Publishing (Similarity: 83%)
  - Similar: DK Publishing (Similarity: 81%)
  - Similar: Ig Publishing (Similarity: 81%)
  - Similar: Deodand Publishing (Similarity: 81%)
  - Similar: Orion Publishing (Similarity: 80%)
  - Similar: Delta Publishing (Similarity: 80%)
  - Similar: Van Patten Publishing (Similarity: 80%)
Publisher: Revive Publishing
  - Similar: Regnery Publishing (Similarity: 80%)
Publisher: Bloomsbury Children's Books
  - Similar: Bloomsbury Childrens Books (Similarity: 98%)
  - Similar: Bloomsbury USA Children's Books (Similarity: 93%)
  - Similar: Bloomsbury U.S.A. Children's Books (Similarity: 90%)
  - Similar: Bloomsbury USA Childrens (Similarity: 82%)
Publisher: Marion Boyars Publishers Ltd
  - Similar: Marion Boyars Publishers (Similarity: 92%)
Publisher: Deutscher Taschenbuch Verlag
  - Similar: Fischer Taschenbuch Verlag (Similarity: 89%)
Publisher: University of Utah Press
  - Similar: Oxford University Press  USA (Similarity: 86%)
  - Similar: Oxford University Press (Similarity: 81%)
  - Similar: Yale University Press (Similarity: 80%)
Publisher: BBC Audiobooks Ltd
  - Similar: BBC Audiobooks (Similarity: 88%)
Publisher: ACE
  - Similar: Ace (Similarity: 100%)
Publisher: Bloomsbury U.S.A. Children's Books
  - Similar: Bloomsbury USA Children's Books (Similarity: 91%)
  - Similar: Bloomsbury Children's Books (Similarity: 90%)
  - Similar: Bloomsbury Childrens Books (Similarity: 88%)
Publisher: University Press of America
  - Similar: Dante University of America Press (Similarity: 90%)
  - Similar: University of Georgia Press (Similarity: 89%)
  - Similar: University of Arizona Press (Similarity: 89%)
  - Similar: University of Michigan Press (Similarity: 87%)
  - Similar: University of Chicago Press (Similarity: 85%)
  - Similar: University Of Chicago Press (Similarity: 85%)
  - Similar: American University in Cairo Press (Similarity: 85%)
  - Similar: University Press of Florida (Similarity: 85%)
  - Similar: University of California Press (Similarity: 84%)
  - Similar: University of Arkansas Press (Similarity: 84%)
  - Similar: University of Notre Dame Press (Similarity: 84%)
  - Similar: University of New Mexico Press (Similarity: 84%)
  - Similar: University Press of Kansas (Similarity: 83%)
  - Similar: University of Alaska Press (Similarity: 83%)
  - Similar: University of Minnesota Press (Similarity: 82%)
  - Similar: University Of Minnesota Press (Similarity: 82%)
  - Similar: University of Calgary Press (Similarity: 81%)
  - Similar: University Press of Kentucky (Similarity: 80%)
  - Similar: University of Nebraska Press (Similarity: 80%)
Publisher: University of Calgary Press
  - Similar: University of Alaska Press (Similarity: 87%)
  - Similar: University of Chicago Press (Similarity: 85%)
  - Similar: University Of Chicago Press (Similarity: 85%)
  - Similar: University of California Press (Similarity: 84%)
  - Similar: University of Michigan Press (Similarity: 84%)
  - Similar: University Press of Kansas (Similarity: 83%)
  - Similar: University of North Carolina Press (Similarity: 82%)
  - Similar: University Press of America (Similarity: 81%)
  - Similar: University of Arizona Press (Similarity: 81%)
  - Similar: University Press of Kentucky (Similarity: 80%)
  - Similar: University of Arkansas Press (Similarity: 80%)
  - Similar: University of Nebraska Press (Similarity: 80%)
Publisher: Bloomsbury Childrens Books
  - Similar: Bloomsbury Children's Books (Similarity: 98%)
  - Similar: Bloomsbury USA Children's Books (Similarity: 91%)
  - Similar: Bloomsbury U.S.A. Children's Books (Similarity: 88%)
  - Similar: Bloomsbury USA Childrens (Similarity: 80%)
Publisher: MacMillan
  - Similar: Macmillan (Similarity: 100%)
  - Similar: MacMillan UK (Similarity: 86%)
  - Similar: Pan MacMillan (Similarity: 82%)
Publisher: Recorded Books
  - Similar: Recorded Books  Inc. (Similarity: 88%)
Publisher: Henry Holt and Co. (BYR)
  - Similar: Henry Holt and Co. (Similarity: 89%)
Publisher: Pi Press
  - Similar: Paris Press (Similarity: 84%)
  - Similar: Oni Press (Similarity: 82%)
  - Similar: MIT Press (Similarity: 82%)
  - Similar: Polity Press (Similarity: 80%)
Publisher: Regan Books
  - Similar: Regal Books (Similarity: 91%)
  - Similar: Urban Books (Similarity: 82%)
  - Similar: Creation Books (Similarity: 80%)
  - Similar: Reaktion Books (Similarity: 80%)
  - Similar: Pan Books (Similarity: 80%)
Publisher: Emece Editores
  - Similar: Emecé Editores (Similarity: 96%)
Publisher: Bloomsbury USA Children's Books
  - Similar: Bloomsbury Children's Books (Similarity: 93%)
  - Similar: Bloomsbury U.S.A. Children's Books (Similarity: 91%)
  - Similar: Bloomsbury Childrens Books (Similarity: 91%)
  - Similar: Bloomsbury USA Childrens (Similarity: 87%)
Publisher: Harper Perennial (HarperCollins)
  - Similar: HarperCollins Perennial (Similarity: 87%)
  - Similar: William Morrow / HarperCollins / Harper Perennial (Similarity: 80%)
Publisher: Audiogo
  - Similar: AudioGO (Similarity: 100%)
Publisher: Samuel French Ltd
  - Similar: Samuel French  Inc. (Similarity: 82%)
Publisher: The Overlook Press
  - Similar: Overlook Press (Similarity: 88%)
Publisher: Hard Press
  - Similar: Harbor Press (Similarity: 82%)
Publisher: Tom Doherty Associates
  - Similar: Doherty  Tom Associates  LLC (Similarity: 92%)
  - Similar: Tom Doherty Associates/Orb Books (Similarity: 81%)
Publisher: Baker Books
  - Similar: Baen Books (Similarity: 86%)
  - Similar: Ace Books (Similarity: 80%)
Publisher: Oregon State University Press
  - Similar: Kent State University Press (Similarity: 89%)
  - Similar: Ohio State University Press (Similarity: 89%)
  - Similar: Kent State University Press (OH) (Similarity: 81%)
Publisher: Digital Manga Publishing / Dark Horse
  - Similar: Digital Manga Publishing (Similarity: 81%)
Publisher: Stanford University Press
  - Similar: Stanford University Press (Stanford  CA) (Similarity: 81%)
Publisher: Arcadia Publishing (SC)
  - Similar: Arcadia Publishing (Similarity: 92%)
  - Similar: Arcade Publishing (Similarity: 84%)
Publisher: Univ Of Minnesota Press
  - Similar: University of Minnesota Press (Similarity: 88%)
  - Similar: University Of Minnesota Press (Similarity: 88%)
Publisher: Skinner House Books
  - Similar: Tin House Books (Similarity: 82%)
Publisher: Kent State University Press
  - Similar: Kent State University Press (OH) (Similarity: 95%)
  - Similar: Oregon State University Press (Similarity: 89%)
  - Similar: Ohio State University Press (Similarity: 85%)
  - Similar: Wayne State University Press (Similarity: 80%)
Publisher: Leisure Books
  - Similar: Weiser Books (Similarity: 80%)
Publisher: Osprey Publishing
  - Similar: Express Publishing (Similarity: 80%)
  - Similar: Berkley Publishing (Similarity: 80%)
  - Similar: Prestel Publishing (Similarity: 80%)
Publisher: Marvel Comics
  - Similar: Marvel Comics Group (Similarity: 81%)
  - Similar: Image Comics (Similarity: 80%)
Publisher: Marvel Comics Group
  - Similar: Marvel Comics (Similarity: 81%)
Publisher: Éditions de L'Olivier
  - Similar: Editions de l'Olivier (Similarity: 98%)
Publisher: Dodo Press
  - Similar: Do-Not Press (Similarity: 82%)
Publisher: Berkley Publishing
  - Similar: Berkley Publishing Group (Similarity: 86%)
  - Similar: Merlin Publishing (Similarity: 80%)
  - Similar: Albury Publishing (Similarity: 80%)
  - Similar: Osprey Publishing (Similarity: 80%)
Publisher: Universe
  - Similar: iUniverse (Similarity: 94%)
Publisher: Library of America (NY)
  - Similar: Library of America (Similarity: 92%)
  - Similar: New American Library (Similarity: 83%)
Publisher: Ace Book
  - Similar: Ace Books (Similarity: 94%)
Publisher: Square One Publishers
  - Similar: Cooper Square Publishers (Similarity: 89%)
Publisher: Disney-Hyperion
  - Similar: Disney Hyperion Books (Similarity: 83%)
Publisher: George Braziller Inc.
  - Similar: George Braziller (Similarity: 89%)
Publisher: Blackstone Publishing
  - Similar: Gemstone Publishing (Similarity: 80%)
Publisher: Honor Books
  - Similar: Condor Books (Similarity: 87%)
  - Similar: Tor Books (Similarity: 80%)
  - Similar: Orb Books (Similarity: 80%)
  - Similar: HQN Books (Similarity: 80%)
Publisher: Northeastern University Press
  - Similar: Northwestern University Press (Similarity: 97%)
  - Similar: Manchester University Press (Similarity: 86%)
  - Similar: Northwestern Univ Press (Similarity: 85%)
Publisher: Jump At The Sun
  - Similar: Jump at the Sun (Similarity: 100%)
Publisher: New Press  The
  - Similar: The New Press (Similarity: 100%)
  - Similar: New Press (Similarity: 82%)
Publisher: Faber & Faber Limited
  - Similar: Faber & Faber Ltd. (Similarity: 88%)
Publisher: Schocken Books
  - Similar: Schocken Books Inc (Similarity: 88%)
  - Similar: Chosen Books (Similarity: 85%)
Publisher: Graphics Press LLC
  - Similar: Graphics Press (Similarity: 88%)
Publisher: Graphics Press
  - Similar: Graphics Press LLC (Similarity: 88%)
Publisher: Ullstein Tb
  - Similar: Ullstein (Similarity: 84%)
Publisher: Dover Publications (NY)
  - Similar: Dover Publications (NYC) (Similarity: 98%)
  - Similar: Dover Publications  Inc. (Similarity: 93%)
  - Similar: Dover Publications (Similarity: 92%)
  - Similar: Lerner Publications (Similarity: 80%)
  - Similar: Dragon Door Publications (Similarity: 80%)
Publisher: Lulu Press
  - Similar: LSU Press (Similarity: 84%)
  - Similar: Laurel Press (Similarity: 82%)
Publisher: New York University Press
  - Similar: State University of New York Press (Similarity: 85%)
Publisher: Penguin Book Limited
  - Similar: Penguin Books Limited (Similarity: 98%)
  - Similar: Penguin Books  Limited (UK) (Similarity: 91%)
  - Similar: Penguin Books Ltd (Similarity: 86%)
  - Similar: Penguin Books Ltd. (Similarity: 86%)
Publisher: Everymans Library
  - Similar: Everyman's Library (Similarity: 91%)
  - Similar: Everyman's Library 234 (Similarity: 82%)
Publisher: Emecé Editores
  - Similar: Emece Editores (Similarity: 96%)
Publisher: Penguin Classics Deluxe Edition
  - Similar: Penguin Classics Deluxe Editions (Similarity: 98%)
Publisher: Pelican Publishing Company
  - Similar: Dell Publishing Company (Similarity: 86%)
  - Similar: Owl Publishing Company (Similarity: 83%)
  - Similar: Pelican Publishing (Similarity: 82%)
  - Similar: MacMillan Publishing Company (Similarity: 81%)
  - Similar: Open Court Publishing Company (Similarity: 80%)
Publisher: Overlook Press
  - Similar: The Overlook Press (Similarity: 88%)
  - Similar: Overlook TP (Similarity: 80%)
Publisher: rbooks
  - Similar: iBooks (Similarity: 83%)
Publisher: Classic Books
  - Similar: Scholastic Books (Similarity: 83%)
Publisher: Laurel Leaf
  - Similar: Dell Laurel-Leaf (Similarity: 81%)
Publisher: Macmillan Publishing Company/Collier Books
  - Similar: MacMillan Publishing Company (Similarity: 80%)
Publisher: Alfred A. Knopf  Inc.
  - Similar: Alfred A. Knopf (Similarity: 88%)
  - Similar: Alfred A.Knopf (Similarity: 88%)
Publisher: State University of New York Press
  - Similar: New York University Press (Similarity: 85%)
  - Similar: Kent State University Press (OH) (Similarity: 81%)
Publisher: Farrar  Strauss & Giroux-3pl
  - Similar: Farrar Straus Giroux (Similarity: 89%)
  - Similar: Farrar  Straus & Giroux (Similarity: 89%)
  - Similar: Farrar  Straus and Giroux (Similarity: 86%)
  - Similar: Farrar Straus and Giroux (Similarity: 86%)
  - Similar: Farrar  Straus and Giroux (NY) (Similarity: 81%)
Publisher: Harvest Books
  - Similar: Pharos Books (Similarity: 80%)
Publisher: National Geographic
  - Similar: National Geographic Society (Similarity: 83%)
Publisher: Spectra/Bantam Books (NYC)
  - Similar: Bantam Spectra (NY) (Similarity: 83%)
Publisher: Bantam Books (NY)
  - Similar: Bantam Books  Inc. (Similarity: 90%)
  - Similar: Bantam Books (Similarity: 89%)
Publisher: Penguin Books (NYC)
  - Similar: Penguin Books (NY) (Similarity: 97%)
  - Similar: Penguin Books (Similarity: 87%)
  - Similar: Penguin Books USA Inc. (Similarity: 84%)
  - Similar: Penguin Books Ltd (Similarity: 82%)
  - Similar: Penguin Books Ltd. (Similarity: 82%)
  - Similar: Penguin Books Canada (Similarity: 81%)
  - Similar: Penguin Books (London) (Similarity: 81%)
Publisher: Fourth Estate Ltd
  - Similar: Fourth Estate (Similarity: 87%)
  - Similar: Fourth Estate (GB) (Similarity: 85%)
Publisher: Jump at the Sun
  - Similar: Jump At The Sun (Similarity: 100%)
Publisher: Hyperio
  - Similar: Hyperion (Similarity: 93%)
Publisher: W.W. Norton & Company  Inc. (NY)
  - Similar: W.W. Norton & Company (NY) (Similarity: 91%)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 89%)
  - Similar: W.W. Norton & Company (NY/London) (Similarity: 87%)
  - Similar: W. W. Norton & Company (Similarity: 84%)
  - Similar: W. W. Norton  Company (Similarity: 84%)
  - Similar: W.W. Norton & Company (Similarity: 84%)
  - Similar: W.W. Norton  Company (Similarity: 84%)
Publisher: The Modern Library New York
  - Similar: The Modern Library (Similarity: 80%)
Publisher: Haus Publishing
  - Similar: AVA Publishing (Similarity: 83%)
  - Similar: Pan Publishing (Similarity: 83%)
  - Similar: Albury Publishing (Similarity: 81%)
  - Similar: Plexus Publishing (Similarity: 81%)
Publisher: University Of Minnesota Press
  - Similar: University of Minnesota Press (Similarity: 100%)
  - Similar: Univ Of Minnesota Press (Similarity: 88%)
  - Similar: University Press of Kansas (Similarity: 84%)
  - Similar: University of Nebraska Press (Similarity: 84%)
  - Similar: University Press of America (Similarity: 82%)
  - Similar: University of Arizona Press (Similarity: 82%)
  - Similar: University of Illinois Press (Similarity: 81%)
  - Similar: University of Michigan Press (Similarity: 81%)
  - Similar: University of Arkansas Press (Similarity: 81%)
  - Similar: University of Notre Dame Press (Similarity: 81%)
  - Similar: University of New Mexico Press (Similarity: 81%)
  - Similar: University of Alaska Press (Similarity: 80%)
Publisher: A Harvest Book/Harcourt  Inc.
  - Similar: Harcourt  Inc.(Harvest Book) (Similarity: 96%)
  - Similar: Harvest Books/Harcourt (Similarity: 86%)
Publisher: Paris Press
  - Similar: Pi Press (Similarity: 84%)
  - Similar: Paulist Press (Similarity: 83%)
Publisher: Overlook TP
  - Similar: Overlook Press (Similarity: 80%)
Publisher: MIT Press (MA)
  - Similar: MIT Press (Similarity: 86%)
  - Similar: Amistad Press (Similarity: 80%)
Publisher: The MIT Press
  - Similar: MIT Press (Similarity: 82%)
Publisher: Polity Press
  - Similar: Clarity Press (Similarity: 80%)
  - Similar: Paulist Press (Similarity: 80%)
  - Similar: Pi Press (Similarity: 80%)
Publisher: John Wiley
  - Similar: John Wiley & Sons (Similarity: 80%)
Publisher: Pan MacMillan
  - Similar: Macmillan (Similarity: 82%)
  - Similar: MacMillan (Similarity: 82%)
  - Similar: MacMillan UK (Similarity: 80%)
Publisher: Bollingen/Princeton University Press (NJ)
  - Similar: Princeton University Press (NJ) (Similarity: 85%)
  - Similar: Princeton University Press (Similarity: 80%)
Publisher: Penguin Audio UK
  - Similar: Penguin Audio (Similarity: 90%)
Publisher: Annick Press
  - Similar: AK Press (Similarity: 80%)
Publisher: Plaza & Janes Editores  S.A.
  - Similar: Plaza & Janés Editores  S.A. (Similarity: 98%)
  - Similar: Plaza & Janes Editores Sa (Similarity: 94%)
Publisher: Arts Publications
  - Similar: Krause Publications (Similarity: 83%)
  - Similar: Dover Publications (Similarity: 80%)
Publisher: Hiperión
  - Similar: Hyperion (Similarity: 80%)
Publisher: Viking Books for Young Readers
  - Similar: HMH Books for Young Readers (Similarity: 81%)
Publisher: Warne
  - Similar: Warner (Similarity: 91%)
Publisher: Amereon Limited
  - Similar: Amereon Ltd (Similarity: 85%)
Publisher: Penguin Books Limited
  - Similar: Penguin Book Limited (Similarity: 98%)
  - Similar: Penguin Books  Limited (UK) (Similarity: 93%)
  - Similar: Penguin Books Ltd (Similarity: 89%)
  - Similar: Penguin Books Ltd. (Similarity: 89%)
Publisher: University of Wisconsin Press
  - Similar: University of Washington Press (Similarity: 81%)
Publisher: W. W. Norton and Company
  - Similar: W. W. Norton & Company (Similarity: 90%)
  - Similar: W. W. Norton  Company (Similarity: 90%)
  - Similar: W.W. Norton & Company (Similarity: 90%)
  - Similar: W.W. Norton  Company (Similarity: 90%)
  - Similar: W.W. Norton & Company (NY) (Similarity: 84%)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 82%)
Publisher: Weiser Books
  - Similar: Wenner Books (Similarity: 83%)
  - Similar: Leisure Books (Similarity: 80%)
Publisher: MacMillan Children's Books
  - Similar: Macmillan Children's Books (Similarity: 100%)
Publisher: Rowohlt Tb.
  - Similar: Rowohlt Tb (Similarity: 100%)
  - Similar: Rowohlt (Similarity: 82%)
Publisher: Open Court Publishing Company
  - Similar: Crossroad Publishing Company (Similarity: 81%)
  - Similar: Pelican Publishing Company (Similarity: 80%)
Publisher: Pocket Books / Simon & Schuster  Inc.
  - Similar: Pocket Books/Simon & Schuster (NY) (Similarity: 95%)
Publisher: Time Life Books
  - Similar: Time-Life Books (Similarity: 100%)
  - Similar: Time-Life Books  Inc. (Similarity: 88%)
Publisher: Penguin Books Ltd.
  - Similar: Penguin Books Ltd (Similarity: 100%)
  - Similar: Penguin Books Limited (Similarity: 89%)
  - Similar: Penguin Books (Similarity: 87%)
  - Similar: Penguin Book Limited (Similarity: 86%)
  - Similar: Penguin Books (London) (Similarity: 86%)
  - Similar: Penguin Books (NY) (Similarity: 85%)
  - Similar: Plume (Penguin Books Ltd) (Similarity: 85%)
  - Similar: Penguin Books Ltd. (London) (Similarity: 83%)
  - Similar: Penguin Books  Limited (UK) (Similarity: 83%)
  - Similar: Penguin Books (NYC) (Similarity: 82%)
  - Similar: Penguin Books Canada (Similarity: 81%)
Publisher: Circlet Press
  - Similar: CRC Press (Similarity: 82%)
Publisher: BBC Books
  - Similar: B Books (Similarity: 88%)
  - Similar: Basic Books (Similarity: 80%)
Publisher: Plexus Publishing
  - Similar: Plexus Publishing Ltd (Similarity: 89%)
  - Similar: Haus Publishing (Similarity: 81%)
  - Similar: Planeta Publishing (Similarity: 80%)
  - Similar: Express Publishing (Similarity: 80%)
Publisher: Simon & Schuster Books for Young Readers
  - Similar: Simon  Schuster Books for Young Readers (Similarity: 100%)
Publisher: Barn Owl Books
  - Similar: Barnes & Noble Books (Similarity: 81%)
Publisher: Zondervan Publishing House
  - Similar: Zondervan Publishing Company (Similarity: 81%)
Publisher: The Berkley Publishing Group
  - Similar: Berkley Publishing Group (Similarity: 92%)
  - Similar: Avery Publishing Group (Similarity: 80%)
Publisher: Ecco / HarperCollins
  - Similar: HarperCollins (Similarity: 84%)
  - Similar: Harpercollins (Similarity: 84%)
  - Similar: Avon HarperCollins (Similarity: 83%)
  - Similar: HarperCollins Espanol (Similarity: 82%)
Publisher: Chelsea House Publishers
  - Similar: Harvest House Publishers (Similarity: 83%)
  - Similar: Arkham House Publishers (Similarity: 81%)
  - Similar: Chelsea House Publications (Similarity: 80%)
Publisher: Orchard Books (NY)
  - Similar: Orchard Books (Similarity: 90%)
Publisher: Sunstone Press
  - Similar: Taunton Press (Similarity: 81%)
Publisher: Nova Audio Books
  - Similar: Naxos Audiobooks (Similarity: 81%)
Publisher: University Press of Kentucky
  - Similar: University Press of Kansas (Similarity: 81%)
  - Similar: University Press of America (Similarity: 80%)
  - Similar: University of Calgary Press (Similarity: 80%)
Publisher: Harcourt  Brace  Jovanovich
  - Similar: Harcourt Brace Jovanovich (Similarity: 100%)
  - Similar: Harcourt Brace Jovanovich (NY) (Similarity: 94%)
  - Similar: Harcourt  Brace Jovanovich/Harvest (Similarity: 86%)
Publisher: CDS Books
  - Similar: Cedar Books (Similarity: 80%)
  - Similar: Lucas Books (Similarity: 80%)
Publisher: dtv
  - Similar: Dtv (Similarity: 100%)
Publisher: Teacher's Pet Publications
  - Similar: Teacher's Pet Publications  Inc. (Similarity: 93%)
Publisher: Hyperion Paperbacks
  - Similar: Orion Paperbacks (Similarity: 86%)
  - Similar: Harper Paperbacks (Similarity: 83%)
  - Similar: HarperPaperbacks (Similarity: 80%)
Publisher: Douglas & McIntyre
  - Similar: Douglas  McIntyre (Similarity: 100%)
Publisher: DC Comics Vertigo
  - Similar: Vertigo (DC Comics) (Similarity: 100%)
Publisher: Disinformation Company
  - Similar: The Disinformation Company (NYC) (Similarity: 85%)
Publisher: The Disinformation Company (NYC)
  - Similar: Disinformation Company (Similarity: 85%)
Publisher: Bradygames
  - Similar: BradyGames (Similarity: 100%)
Publisher: Del Rey Books (NY)
  - Similar: Del Rey Books (Similarity: 90%)
Publisher: Icon Books
  - Similar: Orion Books (Similarity: 86%)
  - Similar: Chosen Books (Similarity: 82%)
  - Similar: Nation Books (Similarity: 82%)
  - Similar: Condor Books (Similarity: 82%)
  - Similar: Onyx Books (Similarity: 80%)
Publisher: Hackett Publishing Company
  - Similar: Hackett Publishing Company  Inc. (Similarity: 93%)
  - Similar: Hackett Publishing Company  Inc. (USA) (Similarity: 87%)
  - Similar: Dell Publishing Company (Similarity: 82%)
Publisher: Naiad Press
  - Similar: Amistad Press (Similarity: 83%)
  - Similar: Aivia Press (Similarity: 82%)
  - Similar: Oni Press (Similarity: 80%)
Publisher: Penguin UK
  - Similar: Penguin (Similarity: 82%)
Publisher: University of Alaska Press
  - Similar: University of Nebraska Press (Similarity: 89%)
  - Similar: University Press of Kansas (Similarity: 88%)
  - Similar: University of Calgary Press (Similarity: 87%)
  - Similar: University of Arkansas Press (Similarity: 85%)
  - Similar: University Press of Florida (Similarity: 83%)
  - Similar: University Press of America (Similarity: 83%)
  - Similar: University of Arizona Press (Similarity: 83%)
  - Similar: University of California Press (Similarity: 82%)
  - Similar: University of Illinois Press (Similarity: 81%)
  - Similar: University of Minnesota Press (Similarity: 80%)
  - Similar: University Of Minnesota Press (Similarity: 80%)
Publisher: Veritas Publishing
  - Similar: Virgin Publishing (Similarity: 80%)
Publisher: Avery Publishing Group
  - Similar: Berkley Publishing Group (Similarity: 87%)
  - Similar: Carlton Publishing Group (Similarity: 83%)
  - Similar: Crown Publishing Group (Similarity: 82%)
  - Similar: B&H Publishing Group (Similarity: 81%)
  - Similar: The Berkley Publishing Group (Similarity: 80%)
  - Similar: W Publishing Group (Similarity: 80%)
Publisher: Wiley Publishing  Inc.
  - Similar: Wiley Publishing (Similarity: 89%)
Publisher: Thames & Hudson
  - Similar: Thames  Hudson (Similarity: 100%)
  - Similar: Thames & Hudson Ltd (Similarity: 87%)
Publisher: Warner
  - Similar: Warne (Similarity: 91%)
Publisher: B&H Publishing Group
  - Similar: W Publishing Group (Similarity: 84%)
  - Similar: Berkley Publishing Group (Similarity: 82%)
  - Similar: Knopf Publishing Group (Similarity: 81%)
  - Similar: Crown Publishing Group (Similarity: 81%)
  - Similar: Avery Publishing Group (Similarity: 81%)
  - Similar: Orion Publishing Group (Similarity: 81%)
  - Similar: Crown Publishing Group (NY) (Similarity: 80%)
Publisher: Courage Books
  - Similar: Corgi Books (Similarity: 83%)
  - Similar: Forge Books (Similarity: 83%)
Publisher: Plaza & Janés
  - Similar: Plaza y Janés (Similarity: 91%)
  - Similar: Plaza y Janes (Similarity: 87%)
Publisher: Penguin Group (USA)
  - Similar: Penguin Group (Similarity: 87%)
Publisher: Urion Press
  - Similar: Xulon Press (Similarity: 82%)
Publisher: Tusquets
  - Similar: TusQuets (Similarity: 100%)
Publisher: Apple Paperbacks (Scholastic)
  - Similar: Scholastic Paperbacks (Similarity: 88%)
Publisher: Scholastic Inc
  - Similar: Scholastic Inc. (Similarity: 100%)
  - Similar: Scholastic  Inc. (Similarity: 100%)
  - Similar: Scholastic (Similarity: 83%)
  - Similar: Hippo/Scholastic (Similarity: 80%)
Publisher: Dial Books
  - Similar: Daw Books (Similarity: 84%)
  - Similar: NAL Books (Similarity: 84%)
  - Similar: DAW Books (Similarity: 84%)
  - Similar: Rodale Books (Similarity: 82%)
  - Similar: Dafina Books (Similarity: 82%)
  - Similar: Mira Books (Similarity: 80%)
  - Similar: Seal Books (Similarity: 80%)
  - Similar: Mira books (Similarity: 80%)
  - Similar: Gaia Books (Similarity: 80%)
Publisher: Candlewick Press (MA)
  - Similar: Candlewick Press (Similarity: 91%)
Publisher: HarperCollinsPublishers
  - Similar: HarperCollins Publishers (Similarity: 98%)
  - Similar: HarperCollins Publishers  Inc. (Similarity: 90%)
  - Similar: HarperCollins Publishers Ltd (Similarity: 90%)
  - Similar: Collins Publishers (Similarity: 83%)
Publisher: Alderac Entertainment Group (AEG)
  - Similar: Alderac Entertainment Group (Similarity: 93%)
Publisher: Hippo/Scholastic
  - Similar: Scholastic Inc. (Similarity: 80%)
  - Similar: Scholastic  Inc. (Similarity: 80%)
  - Similar: Scholastic Inc (Similarity: 80%)
Publisher: University of Arkansas Press
  - Similar: University Press of Kansas (Similarity: 96%)
  - Similar: University of Alaska Press (Similarity: 85%)
  - Similar: University Press of America (Similarity: 84%)
  - Similar: University of Arizona Press (Similarity: 84%)
  - Similar: University of Nebraska Press (Similarity: 82%)
  - Similar: University of Calgary Press (Similarity: 80%)
Publisher: Listening Library
  - Similar: Listening Library (Audio) (Similarity: 85%)
Publisher: Harpercollins
  - Similar: HarperCollins (Similarity: 100%)
  - Similar: HarperCollins UK (Similarity: 90%)
  - Similar: Ecco / HarperCollins (Similarity: 84%)
  - Similar: Avon HarperCollins (Similarity: 84%)
Publisher: Cleis Press
  - Similar: LSU Press (Similarity: 80%)
Publisher: Laurel Leaf Books
  - Similar: Laurel-Leaf Books (Similarity: 100%)
Publisher: Harper Teen
  - Similar: HarperTeen (Similarity: 95%)
  - Similar: Harpperen (Similarity: 80%)
Publisher: Firebird Books
  - Similar: Firebrand Books (Similarity: 90%)
  - Similar: Fireside Books (Similarity: 86%)
Publisher: Henry Holt and Company
  - Similar: Henry Holt & Company (Similarity: 90%)
  - Similar: Henry Holt and Co. (Similarity: 87%)
Publisher: Time Warner Books UK
  - Similar: TIme Warner Books (Similarity: 92%)
Publisher: Atlantic Books (UK)
  - Similar: Atlantic Books (Similarity: 90%)
  - Similar: North Atlantic Books (Similarity: 81%)
Publisher: Bantam Doubleday Dell Publishing Company Inc.
  - Similar: Bantam Doubleday Dell Publishing Group (Similarity: 80%)
Publisher: Editions de l'Olivier
  - Similar: Éditions de L'Olivier (Similarity: 98%)
Publisher: Creation Books
  - Similar: Reaktion Books (Similarity: 93%)
  - Similar: Nation Books (Similarity: 85%)
  - Similar: Clarion Books (Similarity: 81%)
  - Similar: Regan Books (Similarity: 80%)
  - Similar: Orion Books (Similarity: 80%)
Publisher: Marion Boyars Publishers
  - Similar: Marion Boyars Publishers Ltd (Similarity: 92%)
Publisher: Tuttle Publishing
  - Similar: Sutton Publishing (Similarity: 82%)
Publisher: Wadsworth Publishing Company
  - Similar: Workman Publishing Company (Similarity: 81%)
Publisher: Regal Books
  - Similar: Regan Books (Similarity: 91%)
  - Similar: Seal Books (Similarity: 86%)
  - Similar: NAL Books (Similarity: 80%)
Publisher: LSU Press
  - Similar: Lulu Press (Similarity: 84%)
  - Similar: Lyons Press (Similarity: 80%)
  - Similar: Cleis Press (Similarity: 80%)
  - Similar: Pluto Press (Similarity: 80%)
Publisher: Routledge Taylor & Francis Books Ltd imprint
  - Similar: Routledge/Taylor & Francis Ltd. (Similarity: 80%)
Publisher: Dominion Press
  - Similar: Odonian Press (Similarity: 81%)
Publisher: Penguin Books USA Inc.
  - Similar: Penguin Books (NYC) (Similarity: 84%)
  - Similar: Penguin Books (NY) (Similarity: 81%)
Publisher: University of Notre Dame Press
  - Similar: Dante University of America Press (Similarity: 86%)
  - Similar: University Press of America (Similarity: 84%)
  - Similar: University of New Mexico Press (Similarity: 83%)
  - Similar: University of Minnesota Press (Similarity: 81%)
  - Similar: University Of Minnesota Press (Similarity: 81%)
Publisher: Lark Books
  - Similar: Luna Books (Similarity: 80%)
Publisher: Alderac Entertainment Group
  - Similar: Alderac Entertainment Group (AEG) (Similarity: 93%)
Publisher: Routledge/Taylor & Francis Ltd.
  - Similar: Routledge Taylor & Francis Books Ltd imprint (Similarity: 80%)
Publisher: Sentient Publications
  - Similar: Servant Publications (Similarity: 83%)
Publisher: Knopf Doubleday Publishing Group
  - Similar: Anchor Books/Knopf Doubleday Publishing Group (Similarity: 83%)
  - Similar: Knopf Publishing Group (Similarity: 81%)
  - Similar: Doubleday Publishing (NY) (Similarity: 80%)
Publisher: Random House (Vintage)
  - Similar: Random House Vintage (Similarity: 100%)
  - Similar: Vintage/Random House (NY) (Similarity: 93%)
  - Similar: Random House Vintage Books (Similarity: 87%)
  - Similar: Random House Trade (Similarity: 84%)
  - Similar: Vintage/Random House Inc. (NYC) (Similarity: 83%)
Publisher: Mira
  - Similar: MIRA (Similarity: 100%)
Publisher: Island Books
  - Similar: New Island Books (Similarity: 86%)
  - Similar: Villard Books (Similarity: 80%)
Publisher: Pharos Books
  - Similar: Harvest Books (Similarity: 80%)
Publisher: Fleming H. Revell Company
  - Similar: Fleming H Revell Co (Similarity: 88%)
Publisher: Mira books
  - Similar: Mira Books (Similarity: 100%)
  - Similar: Miramax Books (Similarity: 87%)
  - Similar: Dial Books (Similarity: 80%)
  - Similar: Gaia Books (Similarity: 80%)
Publisher: Holiday House
  - Similar: Hay House (Similarity: 82%)
  - Similar: Holloway House (Similarity: 81%)
Publisher: Loeb Classical Library 443
  - Similar: Loeb Classical Library (Similarity: 92%)
  - Similar: Loeb Classical Library 307 (Similarity: 92%)
Publisher: Harvest Books/Harcourt
  - Similar: Harcourt  Inc.(Harvest Book) (Similarity: 89%)
  - Similar: A Harvest Book/Harcourt  Inc. (Similarity: 86%)
  - Similar: Harvest / Harcourt (Similarity: 84%)
Publisher: Pluto Press
  - Similar: LSU Press (Similarity: 80%)
Publisher: Center Point
  - Similar: Counterpoint (Similarity: 83%)
Publisher: Prentice Hall PTR
  - Similar: Prentice Hall Press (Similarity: 89%)
  - Similar: Prentice Hall (Similarity: 87%)
Publisher: Time-Life Books
  - Similar: Time Life Books (Similarity: 100%)
  - Similar: Time-Life Books  Inc. (Similarity: 88%)
Publisher: Hodder and Stoughton
  - Similar: Hodder & Stoughton (Similarity: 89%)
  - Similar: Hodder & Stoughton Ltd (Similarity: 80%)
Publisher: Howard Books
  - Similar: Orchard Books (Similarity: 80%)
Publisher: Tokyopop
  - Similar: TokyoPop (Similarity: 100%)
Publisher: Worth Publishers
  - Similar: Rockport Publishers (Similarity: 80%)
Publisher: Tin House Books
  - Similar: Skinner House Books (Similarity: 82%)
  - Similar: Chosen Books (Similarity: 81%)
Publisher: Farrar  Straus & Giroux
  - Similar: Farrar Straus Giroux (Similarity: 100%)
  - Similar: Farrar  Straus and Giroux (Similarity: 91%)
  - Similar: Farrar Straus and Giroux (Similarity: 91%)
  - Similar: Farrar  Strauss & Giroux-3pl (Similarity: 89%)
  - Similar: Farrar  Straus and Giroux (NY) (Similarity: 85%)
  - Similar: Farrar  Straus and Giroux (BYR) (Similarity: 83%)
Publisher: Kregel Publications
  - Similar: Krause Publications (Similarity: 84%)
Publisher: Harvest / Harcourt
  - Similar: Harvest Books/Harcourt (Similarity: 84%)
Publisher: Farrar  Straus and Giroux (BYR)
  - Similar: Farrar  Straus and Giroux (Similarity: 92%)
  - Similar: Farrar Straus and Giroux (Similarity: 92%)
  - Similar: Farrar  Straus and Giroux (NY) (Similarity: 87%)
  - Similar: Farrar Straus Giroux (Similarity: 83%)
  - Similar: Farrar  Straus & Giroux (Similarity: 83%)
Publisher: Viking UK
  - Similar: Viking (Similarity: 80%)
Publisher: Editions Gallimard
  - Similar: Gallimard Education (Similarity: 86%)
Publisher: Alianza Editorial
  - Similar: Editorial Diana (Similarity: 88%)
Publisher: Gollehon Books
  - Similar: Golden Books (Similarity: 85%)
Publisher: HarperCollins (SanFrancisco)
  - Similar: HarperSanFrancisco (Similarity: 82%)
Publisher: Hackett Publishing Company (Indianapolis  IN)
  - Similar: Hackett Publishing Company  Inc. (Similarity: 81%)
Publisher: University of Nebraska Press
  - Similar: University of Alaska Press (Similarity: 89%)
  - Similar: University Press of Kansas (Similarity: 85%)
  - Similar: University of Georgia Press (Similarity: 84%)
  - Similar: University of Minnesota Press (Similarity: 84%)
  - Similar: University Press of America (Similarity: 84%)
  - Similar: University Of Minnesota Press (Similarity: 84%)
  - Similar: University of Arkansas Press (Similarity: 82%)
  - Similar: University Press of Florida (Similarity: 80%)
  - Similar: University of Arizona Press (Similarity: 80%)
Publisher: Princeton University Press (NJ)
  - Similar: Princeton University Press (Similarity: 95%)
  - Similar: Bollingen/Princeton University Press (NJ) (Similarity: 85%)
Publisher: Ohio State University Press
  - Similar: Oregon State University Press (Similarity: 89%)
  - Similar: Kent State University Press (OH) (Similarity: 88%)
  - Similar: Kent State University Press (Similarity: 85%)
  - Similar: University of Texas Press (Similarity: 81%)
  - Similar: Wayne State University Press (Similarity: 80%)
Publisher: Penguin Books (London)
  - Similar: Penguin Books Ltd. (London) (Similarity: 91%)
  - Similar: Penguin Books Ltd (Similarity: 86%)
  - Similar: Penguin Books Ltd. (Similarity: 86%)
  - Similar: Penguin Books (NY) (Similarity: 83%)
  - Similar: Penguin Books (NYC) (Similarity: 81%)
  - Similar: Penguin Books Canada (Similarity: 80%)
Publisher: Disney Hyperion Books
  - Similar: Disney-Hyperion (Similarity: 83%)
  - Similar: Hyperion Books (Similarity: 80%)
Publisher: The Chicken House
  - Similar: Chicken House (Similarity: 87%)
Publisher: Chicken House
  - Similar: The Chicken House (Similarity: 87%)
Publisher: Onyx Books
  - Similar: Icon Books (Similarity: 80%)
Publisher: Laurel Press
  - Similar: Lulu Press (Similarity: 82%)
Publisher: McGraw-Hill Professional Publishing
  - Similar: McGraw-Hill Medical Publishing (Similarity: 83%)
Publisher: Simon & Schuster Aladddin Paperbacks
  - Similar: Simon & Schuster Paperbacks (Similarity: 85%)
Publisher: Chatto and Windus
  - Similar: Chatto & Windus (Similarity: 87%)
  - Similar: Chatto  Windus (Similarity: 87%)
Publisher: HarperVoyager
  - Similar: Harper Voyager (Similarity: 96%)
Publisher: Fischer (Tb.)
  - Similar: Fischer TB (Similarity: 100%)
Publisher: DAW Books Inc
  - Similar: DAW Books (NY) (Similarity: 88%)
  - Similar: Daw Books (Similarity: 82%)
  - Similar: DAW Books (Similarity: 82%)
  - Similar: Dafina Books (Similarity: 80%)
Publisher: Dante University of America Press
  - Similar: University Press of America (Similarity: 90%)
  - Similar: University of New Mexico Press (Similarity: 86%)
  - Similar: University of Michigan Press (Similarity: 82%)
  - Similar: American University in Cairo Press (Similarity: 81%)
  - Similar: University Press of Florida (Similarity: 80%)
  - Similar: University of Arizona Press (Similarity: 80%)
Publisher: William Morrow / HarperCollins / Harper Perennial
  - Similar: Perennial / William Morrow / HarperCollins (Similarity: 92%)
  - Similar: Harper Perennial (HarperCollins) (Similarity: 80%)
Publisher: Van Patten Publishing
  - Similar: Pan Publishing (Similarity: 80%)
Publisher: New York Review Children's Collection
  - Similar: The New York Review Children's Collection (Similarity: 95%)
Publisher: TSR Inc./Wizards of the Coast
  - Similar: Wizards of the Coast (Similarity: 83%)
Publisher: Sutton Publishing
  - Similar: Tuttle Publishing (Similarity: 82%)
Publisher: Ace Hardcover
  - Similar: DAW Hardcover (Similarity: 85%)
Publisher: Dover Publications (NYC)
  - Similar: Dover Publications (NY) (Similarity: 98%)
  - Similar: Dover Publications  Inc. (Similarity: 95%)
  - Similar: Dover Publications (Similarity: 90%)
Publisher: Panther
  - Similar: Pantheon (Similarity: 80%)
Publisher: Collins Publishers
  - Similar: HarperCollins Publishers (Similarity: 86%)
  - Similar: HarperCollinsPublishers (Similarity: 83%)
  - Similar: Crown Publishers (Similarity: 82%)
Publisher: MIRA
  - Similar: Mira (Similarity: 100%)
Publisher: New World Publishing
  - Similar: New Riders Publishing (Similarity: 83%)
Publisher: Orion Publishing Group  Ltd.
  - Similar: The Orion Publishing Group Ltd (Similarity: 93%)
  - Similar: Orion Publishing Group (Similarity: 92%)
  - Similar: Longman Publishing Group (Similarity: 80%)
  - Similar: Orion Publishing Co. (Similarity: 80%)
  - Similar: Orion Publishing Co (Similarity: 80%)
Publisher: Manchester University Press
  - Similar: Northwestern University Press (Similarity: 86%)
  - Similar: Northeastern University Press (Similarity: 86%)
Publisher: Arrow Books Ltd
  - Similar: Arrow Books (Similarity: 85%)
Publisher: Peter Smith Publisher  Inc.
  - Similar: Peter Smith Publisher (Similarity: 91%)
Publisher: Image Comics
  - Similar: Marvel Comics (Similarity: 80%)
Publisher: Harpercollins Childrens Books
  - Similar: Harpercollins Children's Books (Similarity: 95%)
Publisher: Wheeler Publishing
  - Similar: Wiley Publishing (Similarity: 82%)
Publisher: Fantagraphics
  - Similar: Fantagraphics Books (Similarity: 81%)
Publisher: McGraw-Hill Medical Publishing
  - Similar: McGraw-Hill Professional Publishing (Similarity: 83%)
Publisher: Blanvalet Taschenbuch Verlag
  - Similar: Blanvalet Taschenbuch (Similarity: 86%)
Publisher: International Publishers (NYC)
  - Similar: International Publishers (Similarity: 92%)
Publisher: Ivan R. Dee  Publisher
  - Similar: Ivan R. Dee Publisher (Similarity: 100%)
Publisher: Shepard Publications
  - Similar: Re/Search Publications (Similarity: 81%)
  - Similar: Servant Publications (Similarity: 80%)
Publisher: Servant Publications
  - Similar: Sentient Publications (Similarity: 83%)
  - Similar: Shepard Publications (Similarity: 80%)
Publisher: Urban Books
  - Similar: Regan Books (Similarity: 82%)
  - Similar: Orb Books (Similarity: 80%)
  - Similar: Pan Books (Similarity: 80%)
Publisher: Pocket Books/Star Trek
  - Similar: Pocket Star Books (Similarity: 87%)
Publisher: New Press (NY)
  - Similar: New Press (Similarity: 86%)
Publisher: Seal Press
  - Similar: Specialty Press (Similarity: 80%)
Publisher: Orion Books
  - Similar: Oberon Books (Similarity: 87%)
  - Similar: Icon Books (Similarity: 86%)
  - Similar: Clarion Books (Similarity: 83%)
  - Similar: Corgi Books (Similarity: 82%)
  - Similar: Orbis Books (Similarity: 82%)
  - Similar: Orbit Books (Similarity: 82%)
  - Similar: Tor Books (Similarity: 80%)
  - Similar: Orb Books (Similarity: 80%)
  - Similar: Hyperion Books (Similarity: 80%)
  - Similar: Creation Books (Similarity: 80%)
  - Similar: Reaktion Books (Similarity: 80%)
Publisher: Avenel Books
  - Similar: Avon Books (Similarity: 82%)
  - Similar: Baen Books (Similarity: 82%)
Publisher: McClelland & Stewart
  - Similar: McClelland and Stewart (Similarity: 90%)
Publisher: Editorial Diana
  - Similar: Alianza Editorial (Similarity: 88%)
Publisher: Planeta
  - Similar: Geoplaneta (Similarity: 82%)
Publisher: Modern Library Classics
  - Similar: Loeb Classical Library (Similarity: 80%)
Publisher: Book Publishing Company (TN)
  - Similar: Book Publishing Company (Similarity: 94%)
  - Similar: Abdo Publishing Company (Similarity: 86%)
  - Similar: Western Publishing Company  Golden Books (Similarity: 80%)
Publisher: Book Publishing Company
  - Similar: Book Publishing Company (TN) (Similarity: 94%)
  - Similar: Abdo Publishing Company (Similarity: 91%)
  - Similar: Owl Publishing Company (Similarity: 80%)
Publisher: Specialty Press
  - Similar: Seal Press (Similarity: 80%)
Publisher: Bantem Press
  - Similar: Bantam Press (Similarity: 92%)
Publisher: P & R Publishing
  - Similar: Pan Publishing (Similarity: 86%)
  - Similar: DK Publishing (Similarity: 81%)
  - Similar: Ig Publishing (Similarity: 81%)
  - Similar: W Publishing Group (Similarity: 81%)
  - Similar: Spark Publishing (Similarity: 80%)
  - Similar: B.E.S. Publishing (Similarity: 80%)
Publisher: Green Ronin Publishing
  - Similar: Ronin Publishing (CA) (Similarity: 83%)
Publisher: Harbor Press
  - Similar: Hard Press (Similarity: 82%)
Publisher: William Heinemann Ltd.
  - Similar: William Heinemann (Similarity: 89%)
Publisher: W.W. Norton & Company (NY/London)
  - Similar: W.W. Norton & Company  Inc. (NY) (Similarity: 87%)
  - Similar: W.W. Norton & Company (NY) (Similarity: 86%)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 84%)
Publisher: Sun & Moon Press
  - Similar: Sun and Moon Press (Similarity: 88%)
Publisher: Condor Books
  - Similar: Honor Books (Similarity: 87%)
  - Similar: Icon Books (Similarity: 82%)
Publisher: Ig Publishing
  - Similar: IDW Publishing (Similarity: 89%)
  - Similar: DK Publishing (Similarity: 85%)
  - Similar: Orion Publishing (Similarity: 83%)
  - Similar: AVA Publishing (Similarity: 81%)
  - Similar: Pan Publishing (Similarity: 81%)
  - Similar: P & R Publishing (Similarity: 81%)
  - Similar: Merlin Publishing (Similarity: 80%)
  - Similar: Pippin Publishing (Similarity: 80%)
Publisher: Wesleyan University Press
  - Similar: Yale University Press (Similarity: 83%)
Publisher: Plaza & Janés Editores  S.A.
  - Similar: Plaza & Janes Editores  S.A. (Similarity: 98%)
  - Similar: Plaza & Janes Editores Sa (Similarity: 91%)
Publisher: Rowohlt
  - Similar: Rowohlt Tb (Similarity: 82%)
  - Similar: Rowohlt Tb. (Similarity: 82%)
Publisher: Arkham House Publishers
  - Similar: Bethany House Publishers (Similarity: 81%)
  - Similar: Harvest House Publishers (Similarity: 81%)
Publisher: Health Communications
  - Similar: Health Communications Inc (Similarity: 91%)
  - Similar: Hearst Communications (Similarity: 90%)
Publisher: Puffin Group
  - Similar: Penguin Group (Similarity: 80%)
Publisher: South End Press
  - Similar: South End Press (Boston) (Similarity: 81%)
Publisher: Harcourt  Brace Jovanovich/Harvest
  - Similar: Harcourt Brace Jovanovich (Similarity: 86%)
  - Similar: Harcourt  Brace  Jovanovich (Similarity: 86%)
  - Similar: Harcourt Brace Jovanovich (NY) (Similarity: 82%)
Publisher: Cedar Books
  - Similar: Daw Books (Similarity: 80%)
  - Similar: DAW Books (Similarity: 80%)
  - Similar: CDS Books (Similarity: 80%)
Publisher: Dragon Door Publications
  - Similar: Dover Publications (NY) (Similarity: 80%)
Publisher: Ecco/Harper Perennial
  - Similar: Harper Perennial (Similarity: 86%)
  - Similar: Harper Perennial Classics (Similarity: 83%)
  - Similar: HarperPerennial (Similarity: 83%)
Publisher: Perennial / William Morrow / HarperCollins
  - Similar: William Morrow / HarperCollins / Harper Perennial (Similarity: 92%)
Publisher: Granta Books
  - Similar: Gaia Books (Similarity: 82%)
Publisher: IDW Publishing
  - Similar: DK Publishing (Similarity: 89%)
  - Similar: Ig Publishing (Similarity: 89%)
  - Similar: Dell Publishing (Similarity: 83%)
  - Similar: Orion Publishing (Similarity: 80%)
  - Similar: Delta Publishing (Similarity: 80%)
Publisher: Vintage/Random House (NY)
  - Similar: Random House Vintage (Similarity: 93%)
  - Similar: Random House (Vintage) (Similarity: 93%)
  - Similar: Vintage/Random House Inc. (NYC) (Similarity: 90%)
  - Similar: Random House Vintage Books (Similarity: 82%)
Publisher: HQN Books
  - Similar: Th1nk Books (Similarity: 80%)
  - Similar: Honor Books (Similarity: 80%)
Publisher: HQN
  - Similar: Hqn (Similarity: 100%)
Publisher: Carlton Publishing Group
  - Similar: Crown Publishing Group (Similarity: 91%)
  - Similar: Crown Publishing Group (NY) (Similarity: 86%)
  - Similar: Avery Publishing Group (Similarity: 83%)
Publisher: Ullstein
  - Similar: Ullstein Tb (Similarity: 84%)
Publisher: Serpents Tail
  - Similar: Serpent's Tail (Similarity: 89%)
Publisher: W.W. Norton & Company (NY)
  - Similar: W.W. Norton & Company (NYC) (Similarity: 98%)
  - Similar: W. W. Norton & Company (Similarity: 92%)
  - Similar: W. W. Norton  Company (Similarity: 92%)
  - Similar: W.W. Norton & Company (Similarity: 92%)
  - Similar: W.W. Norton  Company (Similarity: 92%)
  - Similar: W.W. Norton & Company  Inc. (NY) (Similarity: 91%)
  - Similar: W.W. Norton & Company (NY/London) (Similarity: 86%)
  - Similar: W. W. Norton and Company (Similarity: 84%)
Publisher: Readers Digest.
  - Similar: Readers Digest (Similarity: 100%)
Publisher: Readers Digest
  - Similar: Readers Digest. (Similarity: 100%)
Publisher: Krause Publications
  - Similar: Kregel Publications (Similarity: 84%)
  - Similar: Arts Publications (Similarity: 83%)
Publisher: Henry Holt and Co.
  - Similar: Henry Holt and Co. (BYR) (Similarity: 89%)
  - Similar: Henry Holt and Company (Similarity: 87%)
Publisher: The Orion Publishing Group Ltd
  - Similar: Orion Publishing Group  Ltd. (Similarity: 93%)
  - Similar: Orion Publishing Group (Similarity: 85%)
Publisher: Gaia Books
  - Similar: Dafina Books (Similarity: 82%)
  - Similar: Granta Books (Similarity: 82%)
  - Similar: Dial Books (Similarity: 80%)
Publisher: Hartorch
  - Similar: HarperTorch (Similarity: 84%)
Publisher: Wiley Publishing
  - Similar: Wiley Publishing  Inc. (Similarity: 89%)
  - Similar: Wheeler Publishing (Similarity: 82%)
Publisher: Burford Books
  - Similar: Roxford Books (Similarity: 85%)
  - Similar: Bradford Book (Similarity: 85%)
Publisher: Orbit Books
  - Similar: Orbis Books (Similarity: 91%)
  - Similar: Orb Books (Similarity: 90%)
  - Similar: It Books (Similarity: 84%)
  - Similar: Corgi Books (Similarity: 82%)
  - Similar: Orion Books (Similarity: 82%)
  - Similar: Tor Books (Similarity: 80%)
Publisher: Scholastic Audio Books
  - Similar: Scholastic Books (Similarity: 84%)
Publisher: Kaplan Publishing
  - Similar: Pan Publishing (Similarity: 90%)
  - Similar: Planeta Publishing (Similarity: 86%)
  - Similar: Pelican Publishing (Similarity: 86%)
  - Similar: AVA Publishing (Similarity: 84%)
  - Similar: Alliance Publishing (Similarity: 83%)
  - Similar: DK Publishing (Similarity: 80%)
Publisher: Modern Publishing
  - Similar: Merlin Publishing (Similarity: 88%)
  - Similar: Orion Publishing (Similarity: 85%)
  - Similar: Dell Publishing (Similarity: 81%)
  - Similar: Melodrama Publishing (Similarity: 81%)
  - Similar: DK Publishing (Similarity: 80%)
  - Similar: Methuen Publishing (Similarity: 80%)
  - Similar: Deodand Publishing (Similarity: 80%)
Publisher: Berkley Books (NY)
  - Similar: Berkley Books (Similarity: 90%)
Publisher: Lucas Books
  - Similar: Luna Books (Similarity: 86%)
  - Similar: CDS Books (Similarity: 80%)
Publisher: Omnibus Press
  - Similar: Oni Press (Similarity: 82%)
Publisher: Tom Doherty Associates/Orb Books
  - Similar: Tom Doherty Associates (Similarity: 81%)
Publisher: Deodand Publishing
  - Similar: Delta Publishing (Similarity: 82%)
  - Similar: Pan Publishing (Similarity: 81%)
  - Similar: Modern Publishing (Similarity: 80%)
Publisher: Academic Press
  - Similar: Academica Press (Similarity: 97%)
  - Similar: Candlewick Press (Similarity: 80%)
Publisher: Vintage/Random House Inc. (NYC)
  - Similar: Vintage/Random House (NY) (Similarity: 90%)
  - Similar: Random House Vintage (Similarity: 83%)
  - Similar: Random House (Vintage) (Similarity: 83%)
Publisher: Aris & Phillips
  - Similar: Aris and Phillips (Similarity: 87%)
Publisher: Harcourt Brace Jovanovich (NY)
  - Similar: Harcourt Brace Jovanovich (Similarity: 94%)
  - Similar: Harcourt  Brace  Jovanovich (Similarity: 94%)
  - Similar: Harcourt  Brace Jovanovich/Harvest (Similarity: 82%)
Publisher: Harcourt Children's Books
  - Similar: Orion Children's Books (Similarity: 81%)
  - Similar: Harpercollins Children's Books (Similarity: 80%)
Publisher: University Press of New England
  - Similar: University of New Mexico Press (Similarity: 82%)
Publisher: Crossroad Publishing Company
  - Similar: The Crossroad Publishing Company (Similarity: 93%)
  - Similar: Owl Publishing Company (Similarity: 80%)
Publisher: Pippin Publishing
  - Similar: Pan Publishing (Similarity: 84%)
  - Similar: Pelican Publishing (Similarity: 80%)
  - Similar: Ig Publishing (Similarity: 80%)
Publisher: Chatto  Windus
  - Similar: Chatto & Windus (Similarity: 100%)
  - Similar: Chatto and Windus (Similarity: 87%)
Publisher: Fab Press
  - Similar: FT Press (Similarity: 82%)
  - Similar: AK Press (Similarity: 82%)
  - Similar: Apress (Similarity: 80%)
  - Similar: Focal Press (Similarity: 80%)
Publisher: Avon HarperCollins
  - Similar: HarperCollins (Similarity: 84%)
  - Similar: Harpercollins (Similarity: 84%)
  - Similar: Ecco / HarperCollins (Similarity: 83%)
  - Similar: HarperCollins Espanol (Similarity: 82%)
Publisher: Sage Publications Ltd
  - Similar: Sage Publications  Inc (Similarity: 86%)
  - Similar: Let's Go Publications (Similarity: 81%)
Publisher: Elloras Cave
  - Similar: Ellora's Cave (Similarity: 96%)
Publisher: Dtv
  - Similar: dtv (Similarity: 100%)
Publisher: B.E.S. Publishing
  - Similar: P & R Publishing (Similarity: 80%)
Publisher: Zondervan Publishing Company
  - Similar: Workman Publishing Company (Similarity: 85%)
  - Similar: Zondervan Publishing House (Similarity: 81%)
Publisher: Hearst Communications
  - Similar: Health Communications (Similarity: 90%)
  - Similar: Health Communications Inc (Similarity: 83%)
Publisher: Macmillan Children's Books
  - Similar: MacMillan Children's Books (Similarity: 100%)
Publisher: Barnes  Noble
  - Similar: Barnes & Noble (Similarity: 100%)
  - Similar: Barnes & Noble Books (Similarity: 80%)
Publisher: Xulon Press
  - Similar: Urion Press (Similarity: 82%)
Publisher: Holloway House
  - Similar: Holiday House (Similarity: 81%)
Publisher: Reaktion Books
  - Similar: Creation Books (Similarity: 93%)
  - Similar: Nation Books (Similarity: 85%)
  - Similar: Regan Books (Similarity: 80%)
  - Similar: Orion Books (Similarity: 80%)
Publisher: Power Press
  - Similar: Pogue Press (Similarity: 82%)
Publisher: Let's Go Publications
  - Similar: Sage Publications Ltd (Similarity: 81%)
Publisher: Random House Inc.
  - Similar: Random House  Inc. (Similarity: 100%)
  - Similar: Random House (NY) (Similarity: 90%)
  - Similar: Random House (Similarity: 86%)
  - Similar: Random house (Similarity: 86%)
Publisher: Harcourt Brace & Company
  - Similar: Harcourt Brace & company (Similarity: 100%)
Publisher: South End Press (Boston)
  - Similar: South End Press (Similarity: 81%)
Publisher: Random House Reference Publishing
  - Similar: Random House Value Publishing (Similarity: 81%)
Publisher: Vertigo (DC Comics)
  - Similar: DC Comics Vertigo (Similarity: 100%)
Publisher: ECW Press
  - Similar: New Press (Similarity: 89%)
  - Similar: Ecco Press (Similarity: 84%)
  - Similar: CQ Press (Similarity: 82%)
Publisher: Editorial RM
  - Similar: Norma Editorial (Similarity: 89%)
  - Similar: Roca Editorial (Similarity: 85%)
Publisher: Academica Press
  - Similar: Academic Press (Similarity: 97%)
Publisher: University of New Mexico Press
  - Similar: Dante University of America Press (Similarity: 86%)
  - Similar: University Press of America (Similarity: 84%)
  - Similar: University of Michigan Press (Similarity: 83%)
  - Similar: University of Notre Dame Press (Similarity: 83%)
  - Similar: University Press of New England (Similarity: 82%)
  - Similar: University of Chicago Press (Similarity: 81%)
  - Similar: University Of Chicago Press (Similarity: 81%)
  - Similar: University of Minnesota Press (Similarity: 81%)
  - Similar: University Of Minnesota Press (Similarity: 81%)
  - Similar: University of Arizona Press (Similarity: 81%)
Publisher: Clarkson Potter/Publishers
  - Similar: Clarkson Potter Publishers (Similarity: 100%)
Publisher: St. Martin's Paperback
  - Similar: St. Martin's Paperbacks (Similarity: 98%)
Publisher: Rux Martin/Houghton Mifflin Harcourt
  - Similar: Houghton Mifflin Harcourt P (Similarity: 83%)
  - Similar: Houghton Mifflin Harcourt (Similarity: 82%)
Publisher: Berkley Prime Crime Books
  - Similar: Berkley Prime Crime (Similarity: 86%)
Publisher: Berkley Prime Crime
  - Similar: Berkley Prime Crime Books (Similarity: 86%)
Publisher: Texas A&M University Press
  - Similar: University of Texas Press (Similarity: 90%)
  - Similar: Rutgers University Press (Similarity: 80%)
Publisher: Éditions 10/18
  - Similar: Editions 10/18 (Similarity: 96%)
Publisher: Doherty  Tom Associates  LLC
  - Similar: Tom Doherty Associates (Similarity: 92%)
Publisher: Harcourt Brace & company
  - Similar: Harcourt Brace & Company (Similarity: 100%)
Publisher: Running Press Adult
  - Similar: Running Press (Similarity: 81%)
  - Similar: Running Press Kids (Similarity: 81%)
Publisher: Orion Publishing Group
  - Similar: Orion Publishing Group  Ltd. (Similarity: 92%)
  - Similar: Orion Publishing Co. (Similarity: 88%)
  - Similar: Orion Publishing Co (Similarity: 88%)
  - Similar: The Orion Publishing Group Ltd (Similarity: 85%)
  - Similar: Orion Publishing (Similarity: 84%)
  - Similar: Interlink Publishing Group (Similarity: 83%)
  - Similar: Longman Publishing Group (Similarity: 83%)
  - Similar: Knopf Publishing Group (Similarity: 82%)
  - Similar: B&H Publishing Group (Similarity: 81%)
  - Similar: W Publishing Group (Similarity: 80%)
Publisher: Howard University Press
  - Similar: Harvard University Press (Similarity: 89%)
  - Similar: Oxford University Press (Similarity: 87%)
  - Similar: Cambridge University Press (Similarity: 82%)
  - Similar: Cornell University Press (Similarity: 81%)
  - Similar: Oxford University Press  USA (Similarity: 80%)
Publisher: Faber & Faber Ltd.
  - Similar: Faber & Faber Limited (Similarity: 88%)
  - Similar: Faber  Faber (Similarity: 85%)
  - Similar: Faber & Faber (Similarity: 85%)
Publisher: Aivia Press
  - Similar: Naiad Press (Similarity: 82%)
Publisher: CRC Press
  - Similar: Ecco Press (Similarity: 84%)
  - Similar: Circlet Press (Similarity: 82%)
  - Similar: CQ Press (Similarity: 82%)
Publisher: DAW Books (NY)
  - Similar: DAW Books Inc (Similarity: 88%)
  - Similar: Daw Books (Similarity: 86%)
  - Similar: DAW Books (Similarity: 86%)
Publisher: Crown Publishers
  - Similar: Collins Publishers (Similarity: 82%)
Publisher: Dramatists Play Service  Inc.
  - Similar: Dramatists Play Service (Similarity: 92%)
Publisher: Orion Publishing Co
  - Similar: Orion Publishing Co. (Similarity: 100%)
  - Similar: Orion Publishing (Similarity: 91%)
  - Similar: Orion Publishing Group (Similarity: 88%)
  - Similar: Howard Publishing Co (Similarity: 82%)
  - Similar: Orion Publishing Group  Ltd. (Similarity: 80%)
Publisher: Northwestern Univ Press
  - Similar: Northwestern University Press (Similarity: 88%)
  - Similar: Northeastern University Press (Similarity: 85%)
Publisher: CQ Press
  - Similar: ECW Press (Similarity: 82%)
  - Similar: CRC Press (Similarity: 82%)
Publisher: Fawcett crest
  - Similar: Fawcett Crest (Similarity: 100%)
  - Similar: Fawcett Crest Books (Similarity: 81%)
Publisher: Avon Books (P)
  - Similar: Avon Books (Similarity: 91%)
Publisher: Arthur A. Levine Books
  - Similar: Arthur A. Levine (Similarity: 83%)
Publisher: Michael O'Mara Books
  - Similar: Michael O'Mara (Similarity: 82%)
Publisher: Pogue Press
  - Similar: Power Press (Similarity: 82%)
Publisher: University of Arizona Press
  - Similar: University Press of America (Similarity: 89%)
  - Similar: University Press of Kansas (Similarity: 87%)
  - Similar: University of Arkansas Press (Similarity: 87%)
  - Similar: University of Georgia Press (Similarity: 85%)
  - Similar: University Press of Florida (Similarity: 85%)
  - Similar: University of California Press (Similarity: 84%)
  - Similar: University of Alaska Press (Similarity: 83%)
  - Similar: University of South Carolina Press (Similarity: 82%)
  - Similar: University of North Carolina Press (Similarity: 82%)
  - Similar: University of Minnesota Press (Similarity: 82%)
  - Similar: University Of Minnesota Press (Similarity: 82%)
  - Similar: University of Chicago Press (Similarity: 81%)
  - Similar: University Of Chicago Press (Similarity: 81%)
  - Similar: University of Calgary Press (Similarity: 81%)
  - Similar: University of New Mexico Press (Similarity: 81%)
  - Similar: University of Illinois Press (Similarity: 80%)
  - Similar: University of Michigan Press (Similarity: 80%)
  - Similar: University of Nebraska Press (Similarity: 80%)
  - Similar: Dante University of America Press (Similarity: 80%)
Publisher: Hyperion Books for Children
  - Similar: Orion Children's Books (Similarity: 82%)
Publisher: The Audio Partners
  - Similar: Audio Partners (Similarity: 88%)
Publisher: Atlantic Monthly Press (NYC)
  - Similar: Atlantic Monthly Press (Similarity: 92%)
Publisher: Bantam Books  Inc.
  - Similar: Bantam Books (NY) (Similarity: 90%)
  - Similar: Bantam Books (Similarity: 86%)
Publisher: Fischer Taschenbuch Verlag
  - Similar: Deutscher Taschenbuch Verlag (Similarity: 89%)
Publisher: Fischer TB
  - Similar: Fischer (Tb.) (Similarity: 100%)
Publisher: Xlibris Corporation
  - Similar: Limitless Corporation (Similarity: 80%)
Publisher: David C. Cook
  - Similar: David C Cook (Similarity: 100%)
Publisher: W Publishing Group
  - Similar: B&H Publishing Group (Similarity: 84%)
  - Similar: Putnam Publishing Group (Similarity: 83%)
  - Similar: P & R Publishing (Similarity: 81%)
  - Similar: Knopf Publishing Group (Similarity: 80%)
  - Similar: Crown Publishing Group (Similarity: 80%)
  - Similar: Avery Publishing Group (Similarity: 80%)
  - Similar: Orion Publishing Group (Similarity: 80%)
Publisher: Fawcett Crest
  - Similar: Fawcett crest (Similarity: 100%)
  - Similar: Fawcett Crest Books (Similarity: 81%)
Publisher: HarperPaperbacks
  - Similar: Harper Paperbacks (Similarity: 97%)
  - Similar: Hyperion Paperbacks (Similarity: 80%)
Publisher: Transworld Publishers Ltd
  - Similar: Transworld Publishers (Similarity: 91%)
Publisher: Atlas Books/HarperCollins Publishers
  - Similar: HarperCollins Publishers (Similarity: 80%)
Publisher: Hamish hamilton
  - Similar: Hamish Hamilton (Similarity: 100%)
  - Similar: Hamish Hamilton Ltd (Similarity: 88%)
Publisher: Melodrama Publishing
  - Similar: Modern Publishing (Similarity: 81%)
Publisher: Bloomsbury USA Childrens
  - Similar: Bloomsbury USA Children's Books (Similarity: 87%)
  - Similar: Bloomsbury Childrens Books (Similarity: 80%)
Publisher: Fantagraphics Books
  - Similar: Fantagraphics (Similarity: 81%)
Publisher: Middlesex University Press
  - Similar: Cambridge University Press (Similarity: 81%)
Publisher: Pan Books
  - Similar: Bookspan (Similarity: 94%)
  - Similar: Titan Books (Similarity: 80%)
  - Similar: Regan Books (Similarity: 80%)
  - Similar: Urban Books (Similarity: 80%)
Publisher: Do-Not Press
  - Similar: Dodo Press (Similarity: 82%)
Publisher: Modern Library (NY)
  - Similar: Modern Library (Similarity: 90%)
  - Similar: The Modern Library (Similarity: 86%)
Publisher: Rosenberg Publishing
  - Similar: Regnery Publishing (Similarity: 84%)
Publisher: Puffin Audiobooks
  - Similar: Puffin Books (Similarity: 83%)
Publisher: Alyson Books
  - Similar: Avon Books (Similarity: 82%)
Publisher: Hamish Hamilton Ltd
  - Similar: Hamish Hamilton (Similarity: 88%)
  - Similar: Hamish hamilton (Similarity: 88%)
Publisher: St. Martin's
  - Similar: St. Martins Press (Similarity: 81%)
Publisher: William Heinemann
  - Similar: William Heinemann Ltd. (Similarity: 89%)
Publisher: Prestel Publishing
  - Similar: Express Publishing (Similarity: 83%)
  - Similar: Gemstone Publishing (Similarity: 81%)
  - Similar: Osprey Publishing (Similarity: 80%)
  - Similar: Plexus Publishing (Similarity: 80%)
Publisher: Inter-Varsity Press
  - Similar: Inter Varsity Press (Similarity: 100%)
Publisher: DEBOLS!LLO
  - Similar: Debolsillo (Similarity: 90%)
  - Similar: DeBolsillo (Similarity: 90%)